In [12]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
model_id = 'gpt2-large'
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [13]:
from datasets import load_dataset
test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')

Reusing dataset wikitext (/Users/landii03/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors


In [18]:
neg_log_likelihood = outputs[0] * trg_len

In [24]:
neg_log_likelihood

tensor(1621.4019)

In [26]:
nlls

[tensor(1523.1713), tensor(992.5504), tensor(1621.4019)]

In [27]:
encodings.input_ids.size(1)

287644

In [33]:
encodings.input_ids.size()

torch.Size([1, 287644])

In [36]:
input_ids.size()

torch.Size([1, 1024])

In [40]:
len([idx for idx in target_ids[0] if int(idx) != -100])

512

In [41]:
nlls

[tensor(1523.1713), tensor(992.5504), tensor(1621.4019)]

In [14]:
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 512

nlls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i    # may be different from stride on last loop
    input_ids = encodings.input_ids[:,begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)

  1%|          | 3/562 [00:57<2:57:22, 19.04s/it]


KeyboardInterrupt: 

In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [1]:
# from n2c2_dataset import N2c2Dataset
from datasets import load_dataset
from transformers import BertForPreTraining, BertTokenizer, AutoTokenizer
import torch
import torch.nn as nn
import random
from torch.nn import functional as F
from collections import namedtuple
from collections import OrderedDict
from datasets import Dataset, load_dataset
import pickle as pkl
import utils as ut

mydata = load_dataset('./datasets/n2c2_datasets')
MaskedLmInstance = namedtuple("MaskedLmInstance",
                              ["index", "label"])
checkpoint = ut.checkpoint

class TrainingInstance(object):
    """A single training instance (sentence pair)."""

    def __init__(self,
                 tokens,
                 segment_ids,
                 masked_lm_positions,
                 masked_lm_labels,
                 is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

    def __str__(self):
        s = ""
        s += "tokens: %s\n" % (" ".join(
            [x for x in self.tokens]))
        s += "segment_ids: %s\n" % (" ".join([str(x) for x in self.segment_ids]))
        s += "is_random_next: %s\n" % self.is_random_next
        s += "masked_lm_positions: %s\n" % (" ".join(
            [str(x) for x in self.masked_lm_positions]))
        s += "masked_lm_labels: %s\n" % (" ".join(
            [x for x in self.masked_lm_labels]))
        s += "\n"
        return s

    def __repr__(self):
        return self.__str__()

No config specified, defaulting to: n2c2_dataset/language_model
Reusing dataset n2c2_dataset (/Users/landii03/.cache/huggingface/datasets/n2c2_dataset/language_model/0.0.1/34ccee62d89a9c090d608a6043eea469e8600904f5c66bad48e60cfd8122b379)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
rng = random.Random(42)

input_dataset=mydata['train']
max_seq_length=32
dupe_factor=1
short_seq_prob=0.1
masked_lm_prob=0.15 
max_predictions_per_seq=1

vocab_words = list(tokenizer.vocab.keys())
documents = {}
# Iterate over Dataset objects
for el in input_dataset:
    documents.setdefault(el['document'], list()).append(el['sentence'])
instances = []

# rng.shuffle(documents.items())
# for _ in range(dupe_factor):
#     for idx in documents.keys():
#         instances.extend(
#             create_instances_from_document(
#                 documents, idx, max_seq_length, short_seq_prob,
#                 masked_lm_prob, max_predictions_per_seq, vocab_words, rng))

# rng.shuffle(instances)

In [3]:
# Account for [CLS], [SEP], [SEP]
max_num_tokens = max_seq_length - 3

# We *usually* want to fill up the entire sequence since we are padding
# to `max_seq_length` anyways, so short sequences are generally wasted
# computation. However, we *sometimes*
# (i.e., short_seq_prob == 0.1 == 10% of the time) want to use shorter
# sequences to minimize the mismatch between pre-training and fine-tuning.
# The `target_seq_length` is just a rough target however, whereas
# `max_seq_length` is a hard limit.
target_seq_length = max_num_tokens
if rng.random() < short_seq_prob:
    target_seq_length = rng.randint(2, max_num_tokens)

In [4]:
# We DON'T just concatenate all of the tokens from a document into a long
# sequence and choose an arbitrary split point because this would make the
# next sentence prediction task too easy. Instead, we split the input into
# segments "A" and "B" based on the actual "sentences" provided by the user
# input.
instances = []
current_chunk = []
current_length = 0
all_idx = list(documents.keys())
idx = '757'
# print(all_idx)
i = 0
document = documents[idx]

In [5]:
documents.keys()

dict_keys(['641', '643', '681', '704', '757'])

In [6]:
def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng):
    """Truncates a pair of sequences to a maximum sequence length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        print(total_length)
        if total_length <= max_num_tokens:
            break
        print("TRUNCATING")
        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        print(trunc_tokens)
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if rng.random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()
            
def create_masked_lm_predictions(tokens, masked_lm_prob,
                                 max_predictions_per_seq, vocab_words, rng):
    """Creates the predictions for the masked LM objective."""

    cand_indexes = []
    for (i, token) in enumerate(tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue
        cand_indexes.append(i)

    rng.shuffle(cand_indexes)

    output_tokens = list(tokens)

    num_to_predict = min(max_predictions_per_seq,
                         max(1, int(round(len(tokens) * masked_lm_prob))))

    masked_lms = []
    covered_indexes = set()
    for index in cand_indexes:
        if len(masked_lms) >= num_to_predict:
            break
        if index in covered_indexes:
            continue
        covered_indexes.add(index)
        # BERT default percentages
        masked_token = None
        # 80% of the time, replace with [MASK]
        if rng.random() < 0.8:
            masked_token = "[MASK]"
        else:
            # 10% of the time, keep original
            if rng.random() < 0.5:
                masked_token = tokens[index]
            # 10% of the time, replace with random word
            else:
                masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]

        output_tokens[index] = masked_token

        masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))

    masked_lms = sorted(masked_lms, key=lambda x: x.index)

    masked_lm_positions = []
    masked_lm_labels = []
    for p in masked_lms:
        masked_lm_positions.append(p.index)
        masked_lm_labels.append(p.label)

    return output_tokens, masked_lm_positions, masked_lm_labels

In [7]:
while i < len(document):
    segment = document[i].split(' ')  # each segment is a dictionary w/ sentence from the document
    # print(segment)
    current_chunk.append(segment)
    # print(current_chunk)
    current_length += len(segment)  # Number of tokens?
    # print(current_length)
    if i == len(document) - 1 or current_length >= target_seq_length:
        if current_chunk:
            # `a_end` is how many segments from `current_chunk` go into the `A`
            # (first) sentence.
            a_end = 1
            if len(current_chunk) >= 2:
                print('HERE')
                # print(current_chunk)
                a_end = rng.randint(1, len(current_chunk) - 1)
                # print(a_end)

            tokens_a = []
            for j in range(a_end):
                tokens_a.extend(current_chunk[j])
            # print(tokens_a)
            tokens_b = []
            # Random next
            is_random_next = False
            if len(current_chunk) == 1 or rng.random() < 0.5:
                is_random_next = True
                target_b_length = target_seq_length - len(tokens_a)
                # print(target_b_length)

                # This should rarely go for more than one iteration for large
                # corpora. However, just to be careful, we try to make sure that
                # the random document is not the same as the document
                # we're processing.
                # print([i for i in all_idx if i != idx])
                random_document_index = rng.choice([i for i in all_idx if i != idx])

                random_document = documents[random_document_index]
                random_start = rng.randint(0, len(random_document) - 1)
                for j in range(random_start, len(random_document)):
                    tokens_b.extend(random_document[j].split(' '))
                    if len(tokens_b) >= target_b_length:
                        break
                # print(tokens_b)
                # We didn't actually use these segments so we "put them back" so
                # they don't go to waste.
                num_unused_segments = len(current_chunk) - a_end
                i -= num_unused_segments
            # Actual next
            else:
                is_random_next = False
                for j in range(a_end, len(current_chunk)):
                    tokens_b.extend(current_chunk[j])
            truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng)
            print(tokens_a, tokens_b)
            assert len(tokens_a) >= 1
            assert len(tokens_b) >= 1

            tokens = []
            segment_ids = []
            tokens.append("[CLS]")
            segment_ids.append(0)
            for token in tokens_a:
                tokens.append(token)
                segment_ids.append(0)

            tokens.append("[SEP]")
            segment_ids.append(0)

            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)
            # print(tokens)

            (tokens, masked_lm_positions,
             masked_lm_labels) = create_masked_lm_predictions(
                tokens, masked_lm_prob, max_predictions_per_seq, vocab_words, rng)
            instance = TrainingInstance(
                tokens=tokens,
                segment_ids=segment_ids,
                is_random_next=is_random_next,
                masked_lm_positions=masked_lm_positions,
                masked_lm_labels=masked_lm_labels)
            instances.append(instance)
            current_chunk = []
            current_length = 0
        print(instance)
    i += 1

57
TRUNCATING
['301443520', 'ctmc', '49020928', '448922', '1/11/1990', 'am', 'discharge', 'summary', 'unsigned', 'dis', 'admission', 'date', '01/11/1990', 'report', 'status', 'unsigned', 'discharge', 'date', '01/22/1990', 'history', 'of', 'present', 'illness', 'this', 'is', 'the', 'second', 'stamgibonnole', 'hospital', 'admission', 'for', 'this', '50', 'year', 'old', 'woman', 'with', 'a', 'history', 'of', 'hypertension', 'crest', 'syndrome', 'bilateral', 'dvt', 'status', 'post', 'mi', '1984', 'with', 'recurrent', 'angina', 'in', '1986']
56
TRUNCATING
['ctmc', '49020928', '448922', '1/11/1990', 'am', 'discharge', 'summary', 'unsigned', 'dis', 'admission', 'date', '01/11/1990', 'report', 'status', 'unsigned', 'discharge', 'date', '01/22/1990', 'history', 'of', 'present', 'illness', 'this', 'is', 'the', 'second', 'stamgibonnole', 'hospital', 'admission', 'for', 'this', '50', 'year', 'old', 'woman', 'with', 'a', 'history', 'of', 'hypertension', 'crest', 'syndrome', 'bilateral', 'dvt', 'sta

In [24]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.input_ids = []
        self.attention_mask = []
        self.token_type_ids = []
        self.labels = []
        self.next_sentence_label = []
        for dt in dataset.values():
            self.input_ids.append(dt["input_ids"])
            self.attention_mask.append(dt["attention_mask"])
            self.token_type_ids.append(dt["token_type_ids"])
            self.labels.append(dt["labels"])
            self.next_sentence_label.append(dt["next_sentence_label"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in dataset[idx].items()}
        return item

    def __len__(self):
        return len(self.input_ids)

In [66]:
dataset = {}
for (inst_index, instance) in enumerate(instances):
    print(instance.tokens)
    tokenizer.add_tokens(instance.tokens)
    input_ids = tokenizer.convert_tokens_to_ids(
        instance.tokens)  # update vocab with new words given that the tokenization has been done with the raw split(' ') [to modify]
    input_mask = [1] * len(input_ids)
    segment_ids = list(instance.segment_ids)
    print(max_seq_length)
    print(len(input_ids))
    assert len(input_ids) <= max_seq_length

    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    masked_lm_positions = list(instance.masked_lm_positions)
    masked_lm_ids = tokenizer.convert_tokens_to_ids(instance.masked_lm_labels)
    masked_lm_weights = [1.0] * len(masked_lm_ids)
    labels = [-100] * len(input_ids)
    labels[masked_lm_positions[0]] = masked_lm_ids[0]

    while len(masked_lm_positions) < max_predictions_per_seq:
        masked_lm_positions.append(0)
        masked_lm_ids.append(0)
        masked_lm_weights.append(0.0)

    next_sentence_label = 1 if instance.is_random_next else 0

    features = OrderedDict()
    features["input_ids"] = input_ids
    features["attention_mask"] = input_mask
    features["token_type_ids"] = segment_ids
    features["masked_lm_positions"] = masked_lm_positions
    features["masked_lm_ids"] = masked_lm_ids
    features["masked_lm_weights"] = masked_lm_weights
    features["next_sentence_label"] = [next_sentence_label]
    features["labels"] = labels

    dataset[inst_index] = features
    # dt = MyDataset(dataset)

['[CLS]', '[MASK]', 'history', 'of', 'present', 'illness', 'this', 'is', 'the', 'second', 'stamgibonnole', 'hospital', 'admission', 'for', 'this', '50', 'year', 'old', 'woman', 'with', 'a', 'history', 'of', 'hypertension', 'crest', 'syndrome', 'bilateral', '[SEP]', 'he', 'was', 'edentulous', '[SEP]']
32
32
['[CLS]', 'subtotal', 'occlusion', 'of', 'the', 'rca', 'with', 'a', 'high', 'grade', 'lad', 'lesion', 'the', 'patient', 'has', 'had', '[SEP]', 'referred', 'for', 'laser', 'angiography', 'in', '11/89', 'but', 'it', 'was', 'not', 'done', 'secondasry', 'to', 'extent', '[SEP]']
32
32
['[CLS]', '50%', 'ulcerated', 'lad', '100%', 'circumflex', '100%', 'rca', 'inferior', 'posterior', 'focal', 'inferior', 'apical', 'o.k.', 'ef', '49%', '[SEP]', 'past', 'medical', 'history', '[MASK]', 'for', 'as', 'above', 'hypertension', 'crest', 'scleroderma', 'reynaud', 'esophagitis', 'telangectasia', 'and', '[SEP]']
32
32
['[CLS]', '[MASK]', 'is', 'status', 'post', 'bilateral', 'dvt', 'in', '1960', 'and',

In [4]:
import pickle as pkl

dt, tokenizers = pkl.load(open('./out/prova.pt', 'rb'))

In [5]:
dt

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'token_type_ids', 'masked_lm_positions', 'masked_lm_ids', 'masked_lm_weights', 'next_sentence_label', 'labels', 'note_id'],
        num_rows: 137
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'token_type_ids', 'masked_lm_positions', 'masked_lm_ids', 'masked_lm_weights', 'next_sentence_label', 'labels', 'note_id'],
        num_rows: 139
    })
})

In [6]:
tokenizers

{'train': PreTrainedTokenizerFast(name_or_path='emilyalsentzer/Bio_ClinicalBERT', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 'test': PreTrainedTokenizerFast(name_or_path='emilyalsentzer/Bio_ClinicalBERT', vocab_size=28996, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})}

In [2]:
import torch

dt = torch.load(open('./out/prova.pt', 'rb'))

RuntimeError: Invalid magic number; corrupt file?

In [10]:
from datasets import Dataset
dd = Dataset.from_dict(dt['train'].dataset)

In [14]:
dd['input_ids']

[[101,
  28996,
  1142,
  7791,
  1218,
  1105,
  1108,
  28997,
  1106,
  1103,
  17688,
  6059,
  2585,
  1205,
  1111,
  1748,
  102,
  1113,
  28998,
  1744,
  170,
  2229,
  28999,
  3090,
  2967,
  20557,
  29000,
  1105,
  103,
  29001,
  170,
  102],
 [101,
  1119,
  1108,
  29002,
  102,
  29003,
  8682,
  1127,
  2330,
  2589,
  1111,
  1199,
  7648,
  10496,
  29004,
  17688,
  29005,
  1114,
  170,
  25550,
  29006,
  29007,
  29008,
  22895,
  7413,
  103,
  8886,
  1107,
  1103,
  1268,
  3105,
  102],
 [101,
  1103,
  2229,
  11182,
  1127,
  4441,
  1235,
  29009,
  1120,
  1134,
  1159,
  1152,
  1127,
  1865,
  29010,
  102,
  1104,
  20752,
  8936,
  29011,
  103,
  4680,
  170,
  29012,
  29013,
  4680,
  4091,
  29014,
  29015,
  4680,
  29016,
  102],
 [101,
  1126,
  29017,
  3090,
  1126,
  29008,
  13394,
  1104,
  29018,
  102,
  17713,
  29019,
  29020,
  29021,
  122,
  17713,
  29019,
  29020,
  1266,
  1607,
  1103,
  103,
  3756,
  1185,
  1266,
  1607,
 

In [3]:
dt['train'].dataset['note_id']

['681',
 '641',
 '757',
 '704',
 '704',
 '641',
 '643',
 '681',
 '681',
 '757',
 '681',
 '704',
 '643',
 '681',
 '704',
 '643',
 '704',
 '681',
 '641',
 '757',
 '704',
 '641',
 '681',
 '681',
 '757',
 '681',
 '681',
 '704',
 '643',
 '681',
 '643',
 '641',
 '681',
 '757',
 '704',
 '641',
 '643',
 '681',
 '643',
 '643',
 '704',
 '641',
 '643',
 '704',
 '681',
 '681',
 '681',
 '704',
 '641',
 '641',
 '704',
 '681',
 '704',
 '757',
 '641',
 '757',
 '641',
 '643',
 '643',
 '641',
 '643',
 '681',
 '704',
 '681',
 '704',
 '757',
 '757',
 '704',
 '681',
 '704',
 '704',
 '681',
 '641',
 '704',
 '643',
 '757',
 '643',
 '704',
 '704',
 '704',
 '704',
 '643',
 '643',
 '681',
 '681',
 '704',
 '643',
 '704',
 '681',
 '704',
 '704',
 '681',
 '704',
 '704',
 '643',
 '641',
 '704',
 '643',
 '681',
 '681',
 '704',
 '681',
 '643',
 '641',
 '643',
 '641',
 '704',
 '643',
 '704',
 '757',
 '681',
 '704',
 '704',
 '704',
 '704',
 '643',
 '704',
 '704',
 '757',
 '704',
 '643',
 '643',
 '757',
 '681',
 '757',


In [86]:
dt['train'].__getitem__(0)

{'input_ids': tensor([  101, 28996,  1142,  7791,  1218,  1105,  1108, 28997,  1106,  1103,
         17688,  6059,  2585,  1205,  1111,  1748,   102,  1113, 28998,  1744,
           170,  2229, 28999,  3090,  2967, 20557, 29000,  1105,   103, 29001,
           170,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]),
 'next_sentence_label': tensor([1]),
 'labels': tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 20557,  -100,
          -100,  -100])}

In [54]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

train_dataloader = DataLoader(dt, batch_size=2)

for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([2, 32]),
 'attention_mask': torch.Size([2, 32]),
 'token_type_ids': torch.Size([2, 32]),
 'masked_lm_positions': torch.Size([2, 1]),
 'masked_lm_ids': torch.Size([2, 1]),
 'masked_lm_weights': torch.Size([2, 1]),
 'next_sentence_label': torch.Size([2, 1]),
 'labels': torch.Size([2, 32])}

In [55]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForNextSentencePrediction, AutoModel, BertForPreTraining, BertTokenizer
import torch

checkpoint = "emilyalsentzer/Bio_ClinicalBERT"
# checkpoint = 'bert-base-uncased'

old_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer = BertTokenizer.from_pretrained(checkpoint)
# lm_model = AutoModelForMaskedLM.from_pretrained(checkpoint)
# nsp_model = AutoModelForNextSentencePrediction.from_pretrained(checkpoint)
model = BertForPreTraining.from_pretrained(checkpoint)

# model
# def tokenize_function(example):
#     return tokenizer(example['sentence'])

# tkn_dt = mydata.map(tokenize_function)

Some weights of BertForPreTraining were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
len(tokenizer)

29067

In [57]:
len(old_tokenizer)

28996

In [58]:
model.resize_token_embeddings(len(tokenizer))

Embedding(29067, 768)

In [59]:
out = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], token_type_ids=batch['token_type_ids'],
           next_sentence_label=batch['next_sentence_label'], labels=batch['labels'])

In [60]:
out

BertForPreTrainingOutput(loss=tensor(6.4335, grad_fn=<AddBackward0>), prediction_logits=tensor([[[ -5.5433,  -5.7473,  -5.2080,  ...,  -0.2150,   0.6874,   0.4766],
         [ -3.1968,  -2.7779,  -3.8491,  ...,   1.2901,  -0.4080,   0.7658],
         [-10.6632, -12.3199, -10.2557,  ...,   0.5584,  -0.7573,   0.0931],
         ...,
         [ -7.9459,  -8.5186,  -7.6065,  ...,   0.7876,  -0.2686,   0.6793],
         [ -4.4783,  -4.7499,  -5.7213,  ...,   1.5266,   0.6093,   1.1534],
         [ -4.1019,  -4.7829,  -5.0443,  ...,  -0.3381,  -0.3011,  -0.8171]],

        [[ -4.9559,  -5.8952,  -4.3157,  ...,   0.9165,   0.1068,   0.3251],
         [ -5.6556,  -7.7267,  -5.6868,  ...,   3.3330,   0.2633,  -0.9636],
         [ -2.5063,  -5.4581,  -4.3001,  ...,   1.7406,  -0.1276,  -0.1048],
         ...,
         [-10.6349, -11.4123, -11.5160,  ...,   1.5977,  -0.0950,   0.3320],
         [ -3.4796,  -6.6407,  -5.6742,  ...,   0.5350,   1.6468,  -0.1994],
         [ -3.5459,  -4.2667,  -2.0

In [48]:
import numpy as np
logits = out.prediction_logits[1][np.where(batch['labels'][1] != -100)]
predictions = torch.argmax(logits, dim=-1)

In [49]:
tokenizer.convert_ids_to_tokens(predictions)

['lad']

In [50]:
tokenizer.convert_ids_to_tokens(batch['input_ids'][1])

['[CLS]',
 'subtotal',
 'occlusion',
 'of',
 'the',
 'rca',
 'with',
 'a',
 'high',
 'grade',
 'lad',
 'lesion',
 'the',
 'patient',
 'has',
 'had',
 '[SEP]',
 'referred',
 'for',
 'laser',
 'angiography',
 'in',
 '11/89',
 'but',
 'it',
 'was',
 'not',
 'done',
 'secondasry',
 'to',
 'extent',
 '[SEP]']

In [51]:
batch['labels'][1][10]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
        19122,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100])

In [52]:
tokenizer.convert_ids_to_tokens([batch['labels'][1][10]])

['lad']

In [61]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_batch = ["I love Pixar.", "I don't care for Pixar."]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [70]:
encoding

{'input_ids': tensor([[  101,  1045,  2293, 14255, 18684,  2099,  1012,   102,     0,     0,
             0,     0],
        [  101,  1045,  2123,  1005,  1056,  2729,  2005, 14255, 18684,  2099,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [241]:
import torch

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.input_ids = [dt["input_ids"] for dt in dataset.values()]
        # self.input_mask = dataset["input_mask"]
        # self.segment_ids = dataset["segment_ids"]
        # self.masked_lm_positions = dataset["masked_lm_positions"]
        # self.masked_lm_ids = dataset["masked_lm_ids"]
        # self.masked_lm_weights = dataset["masked_lm_weights"]
        # self.next_sentence_labels = dataset["next_sentence_labels"]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in dataset[idx].items()}
        return item

    def __len__(self):
        return len(self.input_ids)

train_dataset = MyDataset(dataset)

In [242]:
train_dataset.__getitem__(0)

{'input_ids': tensor([  101,   103,  1607,  1104,  1675,  6946,  1142,  1110,  1103,  1248,
         28996,  2704, 10296,  1111,  1142,  1851,  1214,  1385,  1590,  1114,
           170,  1607,  1104, 28997, 13468,  9318, 20557,   102,  1119,  1108,
         28998,   102]),
 'input_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]),
 'segment_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 1]),
 'masked_lm_positions': tensor([1]),
 'masked_lm_ids': tensor([100]),
 'masked_lm_weights': tensor([1.]),
 'next_sentence_labels': tensor([1])}

In [244]:
train_dataset.input_ids

[[101,
  103,
  1607,
  1104,
  1675,
  6946,
  1142,
  1110,
  1103,
  1248,
  28996,
  2704,
  10296,
  1111,
  1142,
  1851,
  1214,
  1385,
  1590,
  1114,
  170,
  1607,
  1104,
  28997,
  13468,
  9318,
  20557,
  102,
  1119,
  1108,
  28998,
  102],
 [101,
  28999,
  29000,
  1104,
  1103,
  29001,
  1114,
  170,
  1344,
  3654,
  19122,
  29002,
  1103,
  5351,
  1144,
  1125,
  102,
  2752,
  1111,
  10221,
  29003,
  1107,
  29004,
  1133,
  1122,
  1108,
  1136,
  1694,
  29005,
  1106,
  6102,
  102],
 [101,
  29006,
  29007,
  19122,
  29008,
  29009,
  29008,
  29001,
  15543,
  16530,
  17811,
  15543,
  29010,
  29011,
  29012,
  29013,
  102,
  1763,
  2657,
  1607,
  103,
  1111,
  1112,
  1807,
  28997,
  13468,
  29014,
  29015,
  29016,
  29017,
  1105,
  102],
 [101,
  103,
  1110,
  2781,
  2112,
  20557,
  29018,
  1107,
  2761,
  1105,
  2459,
  5165,
  1114,
  29019,
  1105,
  29020,
  102,
  170,
  1607,
  1104,
  29021,
  1286,
  1981,
  1607,
  1104,
  290

In [ ]:






def write_instance_to_example_files(instances, tokenizer, max_seq_length,
                                    max_predictions_per_seq):
    """Create input data from `TrainingInstance`s."""

    # writers = []
    # for output_file in output_files:
    #     writers.append(tf.compat.v1.python_io.TFRecordWriter(output_file))

    writer_index = 0

    total_written = 0
    for (inst_index, instance) in enumerate(instances):
        print(instance.tokens)
        tokenizer.add_tokens(instance.tokens)
        input_ids = tokenizer.convert_tokens_to_ids(
            instance.tokens)  # update vocab with new words given that the tokenization has been done with the raw split(' ') [to modify]
        input_mask = [1] * len(input_ids)
        segment_ids = list(instance.segment_ids)
        assert len(input_ids) <= max_seq_length

        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        masked_lm_positions = list(instance.masked_lm_positions)
        masked_lm_ids = tokenizer.convert_tokens_to_ids(instance.masked_lm_labels)
        masked_lm_weights = [1.0] * len(masked_lm_ids)

        while len(masked_lm_positions) < max_predictions_per_seq:
            masked_lm_positions.append(0)
            masked_lm_ids.append(0)
            masked_lm_weights.append(0.0)

        next_sentence_label = 1 if instance.is_random_next else 0

        features = dict()
        features["input_ids"] = input_ids
        features["input_mask"] = input_mask
        features["segment_ids"] = segment_ids
        features["masked_lm_positions"] = masked_lm_positions
        features["masked_lm_ids"] = masked_lm_ids
        features["masked_lm_weights"] = masked_lm_weights
        features["next_sentence_labels"] = [next_sentence_label]
        # print(features)
        dataset = Dataset.from_dict(features)
        return dataset
    #
    #     tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    #
    #     writers[writer_index].write(tf_example.SerializeToString())
    #     writer_index = (writer_index + 1) % len(writers)
    #
    #     total_written += 1
    #
    #     if inst_index < 20:
    #         tf.compat.v1.logging.info("*** Example ***")
    #         tf.compat.v1.logging.info("tokens: %s" % " ".join(
    #             [tokenization.printable_text(x) for x in instance.tokens]))
    #
    #         for feature_name in features.keys():
    #             feature = features[feature_name]
    #             values = []
    #             if feature.int64_list.value:
    #                 values = feature.int64_list.value
    #             elif feature.float_list.value:
    #                 values = feature.float_list.value
    #             tf.compat.v1.logging.info(
    #                 "%s: %s" % (feature_name, " ".join([str(x) for x in values])))
    #
    # for writer in writers:
    #     writer.close()
    #
    # tf.compat.v1.logging.info("Wrote %d total instances", total_written)


# model
if __name__ == '__main__':
    

In [26]:
len(mydata['train'][0]['sentence'])

67

In [8]:
import numpy as np
len(np.unique(mydata['train']['document']))
len(np.unique(mydata['train']['challenge']))

7

In [2]:
mydata['test'][]

Dataset({
    features: ['sentence', 'document', 'challenge'],
    num_rows: 297
})

In [10]:
from transformers import AutoTokenizer

In [14]:
class TrainingInstance(AutoTokenizer):
    """A single training instance (sentence pair)."""

    def __init__(self, checkpoint):
        super().__init__()
        super(TrainingInstance, self).from_pretrained(checkpoint)

In [15]:
checkpoint = "emilyalsentzer/Bio_ClinicalBERT"

TrainingInstance(checkpoint)

OSError: AutoTokenizer is designed to be instantiated using the `AutoTokenizer.from_pretrained(pretrained_model_name_or_path)` method.

In [13]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.trainers import WordPieceTrainer

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])
bert_tokenizer.pre_tokenizer = Whitespace()
bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)
trainer = WordPieceTrainer(
    vocab_size=30522, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)
files = [mydata[split]['sentence'] for split in ['train', 'test']]
bert_tokenizer.train(files, trainer)


Exception: No such file or directory (os error 2)

In [15]:
mydata['train']

Dataset({
    features: ['sentence', 'document', 'challenge'],
    num_rows: 5
})

## BERT masked model fine-tuning

In [71]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForNextSentencePrediction, AutoModel, BertForPreTraining, BertTokenizer
import torch
from datasets import load_dataset

checkpoint = "emilyalsentzer/Bio_ClinicalBERT"
# checkpoint = 'bert-base-uncased'
data = load_dataset("glue", "mrpc")

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer = BertTokenizer.from_pretrained(checkpoint)
# lm_model = AutoModelForMaskedLM.from_pretrained(checkpoint)
# nsp_model = AutoModelForNextSentencePrediction.from_pretrained(checkpoint)
model = BertForPreTraining.from_pretrained(checkpoint)

# model
# def tokenize_function(example):
#     return tokenizer(example['sentence'])

# tkn_dt = mydata.map(tokenize_function)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /Users/landii03/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of BertForPreTraining were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [76]:
tkn = tokenizer(data['train']['sentence1'])

In [80]:
data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [78]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], truncation=True)

In [79]:
data.map(tokenize_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

In [284]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

train_dataloader = DataLoader(dt, batch_size=2, collate_fn=data_collator)

for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [287]:
batch['token_type_ids'] = batch['segment_ids']

In [289]:
batch['next_sentence_label'] = batch['next_sentence_labels']

In [290]:
batch

{'input_ids': tensor([[  101,   103,  1607,  1104,  1675,  6946,  1142,   103,  1103,   103,
         28996,  2704,   103,  1111,  1142,  1851,  1214,  1385,  1590,   103,
           170,  1607,  1104, 28997, 13468,  9318, 20557,   102, 20140,  1108,
         28998,   102],
        [  101, 28999, 29000,   103,  1103, 29001,  1114,   170,  1344,  3654,
         19122, 29002,  1103,  5351,  1144,  1125,   102,  2752,  1111,   103,
         29003,  1107, 29004,  1133,  1122,  1108,  1136,  1694, 29005,  1106,
          6102,   102]]), 'input_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'segment_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
     

In [291]:
new_batch = {'input_ids': batch['input_ids'],
            'attention_mask': batch['attention_mask'],
            'token_type_ids': batch['token_type_ids'],
            'labels': batch['labels'],
            'next_sentence_label': batch['next_sentence_label']}

In [300]:
new_ids = []
for tkn in dt.input_ids:
    new_ids.extend(tkn)
tokenizer.add_tokens()

TypeError: add_tokens() missing 1 required positional argument: 'new_tokens'

In [298]:
tokenizer.vocab

{'humorous': 15705,
 'civilian': 6688,
 '1930': 3630,
 'grasped': 16375,
 '##ك': 28494,
 'chronicles': 22329,
 '##smith': 11195,
 'fluids': 24024,
 '21st': 6880,
 'slapping': 25006,
 'momentarily': 17429,
 'gut': 9691,
 'teens': 15050,
 '##chtenstein': 23561,
 'caught': 2347,
 '##ʁ': 28287,
 '##tom': 18778,
 'standings': 12813,
 'describing': 7645,
 'Operational': 24531,
 'Congo': 8695,
 'arches': 14738,
 'Minsk': 24137,
 'matter': 2187,
 '##lman': 12151,
 'enzymes': 17664,
 'attack': 2035,
 '##nat': 24226,
 'boat': 3499,
 'Qaeda': 20205,
 '##に': 28807,
 'Smile': 21278,
 'classic': 5263,
 'vertex': 22132,
 '##lass': 17223,
 'щ': 500,
 '##forcing': 20586,
 '##oxide': 22040,
 '##chet': 17374,
 'vampires': 6039,
 'tiger': 13778,
 'buildings': 2275,
 '₤': 835,
 'Mike': 2639,
 'Mussolini': 25086,
 '##rocodile': 24198,
 'Maid': 27527,
 'martyr': 26899,
 'tuned': 17169,
 'Yes': 2160,
 'landowners': 22671,
 '##stituted': 26742,
 'ladder': 11413,
 'limited': 2609,
 'carnival': 21446,
 '##bon': 

In [292]:
out = model(**new_batch)

IndexError: index out of range in self

In [200]:
import pandas as pd
pd.DataFrame

pyarrow.Table
ciao: int64
ciaociao: int64
----
ciao: [[1,2,3]]
ciaociao: [[4,5,6]]

In [13]:
tokenizer.add_tokens(['contempt', 'supercalifragilistichespiralidoso'])

1

In [14]:
tokenizer.vocab['contempt']

18875

In [15]:
tokenizer.vocab['supercalifragilistichespiralidoso']

28996

In [18]:
import datasets

mydt = datasets.DatasetDict()

In [24]:
mydt['train'] = datasets.Dataset.from_dict({'attention_mask': [1]*10})

In [25]:
mydt

DatasetDict({
    train: Dataset({
        features: ['attention_mask'],
        num_rows: 10
    })
})

In [31]:
notes = []
with open('./out/train_sentences.txt', 'r') as f:
    note_list = f.readlines()
    note = []
    for sen in note_list:
        if sen.strip('\n') == '':
            notes.append(note)
            note = []
        else:
            note.append(sen.strip('\n'))

mynote = notes[0]

sentence1 = 'associated diagnosis urinary tract infection and cirrhosis of the liver'
sentence2 = 'history of present illness the patient is an male who had a history of colon cancer'

In [34]:
tkn = tokenizer(sentence1, sentence2)
tkn
# tokenizer.convert_ids_to_tokens(tkn['input_ids'])

{'input_ids': [101, 2628, 12645, 190, 9324, 1616, 14441, 8974, 1105, 172, 3161, 1197, 15342, 1548, 1104, 1103, 11911, 102, 1607, 1104, 1675, 6946, 1103, 5351, 1110, 1126, 2581, 1150, 1125, 170, 1607, 1104, 1884, 4934, 4182, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [36]:
segment_tensors = torch.tensor([tkn['token_type_ids']])
token_tensors = torch.tensor([tkn['input_ids']])

In [37]:
model(token_tensors, token_type_ids=segment_tensors)

BertForPreTrainingOutput(loss=None, prediction_logits=tensor([[[ -5.0978,  -5.3727,  -5.3744,  ...,  -5.6708,  -6.7929,  -6.1098],
         [ -7.3259,  -7.2716,  -7.7855,  ...,  -6.6673,  -8.9316,  -7.5792],
         [ -5.8150,  -6.0850,  -4.8353,  ...,  -6.9794,  -7.2884,  -6.1053],
         ...,
         [ -7.5736,  -6.1877,  -6.6858,  ...,  -7.5300,  -5.6632,  -7.9293],
         [ -9.7014,  -9.9979, -10.0304,  ...,  -8.5529,  -7.2215,  -8.4779],
         [ -3.5424,  -5.0459,  -4.3914,  ...,  -4.3864,  -6.3197,  -5.1185]]],
       grad_fn=<AddBackward0>), seq_relationship_logits=tensor([[-0.7762,  1.4471]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [1]:
import torch
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer"

# Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)

Using cache found in /Users/landii03/.cache/torch/hub/huggingface_pytorch-transformers_master


In [2]:
tokenizer.convert_ids_to_tokens(indexed_tokens)

['[CLS]',
 'Who',
 'was',
 'Jim',
 'He',
 '##nson',
 '?',
 '[SEP]',
 'Jim',
 'He',
 '##nson',
 'was',
 'a',
 'puppet',
 '##eer',
 '[SEP]']

In [3]:
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, token_type_ids=segments_tensors)

Using cache found in /Users/landii03/.cache/torch/hub/huggingface_pytorch-transformers_master


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
for a, b in zip(tokenizer.convert_ids_to_tokens(indexed_tokens), segments_ids):
    print(a, b)

[CLS] 0
Who 0
was 0
Jim 0
He 0
##nson 0
? 0
[SEP] 0
Jim 1
He 1
##nson 1
was 1
a 1
puppet 1
##eer 1
[SEP] 1


In [6]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
indexed_tokens[masked_index] = tokenizer.mask_token_id
tokens_tensor = torch.tensor([indexed_tokens])

masked_lm_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForMaskedLM', 'bert-base-cased')

with torch.no_grad():
    predictions = masked_lm_model(tokens_tensor, token_type_ids=segments_tensors)

# Get the predicted token
predicted_index = torch.argmax(predictions[0][0], dim=1)[masked_index].item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'Jim'

Using cache found in /Users/landii03/.cache/torch/hub/huggingface_pytorch-transformers_master
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
tokenizer.convert_ids_to_tokens(torch.argmax(predictions[0][0][masked_index]).item())

'Jim'

In [18]:
tokenizer(text_1, text_2)

{'input_ids': [101, 2627, 1108, 3104, 1124, 15703, 136, 102, 3104, 1124, 15703, 1108, 170, 16797, 8284, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
inputs = tokenizer.get_special_tokens_mask(tokenizer(sentence1)['input_ids'])
tokenizer.convert_ids_to_tokens(inputs)

['[unused1]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[unused1]']

In [17]:
true_sent1 = 'associated diagnosis urinary tract infection and cirrhosis of the liver'
labels = tokenizer(true_sent1, sentence2, return_tensors='pt')["input_ids"]
labels

tensor([[  101,  2628, 12645,   190,  9324,  1616, 14441,  8974,  1105,   172,
          3161,  1197, 15342,  1548,  1104,  1103, 11911,   102,  1607,  1104,
          1675,  6946,  1103,  5351,  1110,  1126,  2581,  1150,  1125,   170,
          1607,  1104,  1884,  4934,  4182,   102]])

In [18]:
print(' '.join(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])))
print(' '.join(tokenizer.convert_ids_to_tokens(labels[0])))

[CLS] associated [MASK] u ##rina ##ry tract infection and c ##ir ##r ##hos ##is of the liver [SEP] history of present illness the patient is an male who had a history of co ##lon cancer [SEP]
[CLS] associated diagnosis u ##rina ##ry tract infection and c ##ir ##r ##hos ##is of the liver [SEP] history of present illness the patient is an male who had a history of co ##lon cancer [SEP]


In [19]:
input_tkns = [tkn for tkn in tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])]
print(' '.join(input_tkns))
output_tkns = [tkn for tkn in tokenizer.convert_ids_to_tokens(labels[0])]
print(' '.join(output_tkns))

[CLS] associated [MASK] u ##rina ##ry tract infection and c ##ir ##r ##hos ##is of the liver [SEP] history of present illness the patient is an male who had a history of co ##lon cancer [SEP]
[CLS] associated diagnosis u ##rina ##ry tract infection and c ##ir ##r ##hos ##is of the liver [SEP] history of present illness the patient is an male who had a history of co ##lon cancer [SEP]


In [20]:
outputs = model(**inputs, labels=labels, next_sentence_label=torch.LongTensor([0]))
loss = outputs.loss
lm_logits = outputs.prediction_logits
nsp_logits = outputs.seq_relationship_logits

In [22]:
outputs.prediction_logits.shape

torch.Size([1, 36, 28996])

In [24]:
inputs.input_ids.shape

torch.Size([1, 36])

In [33]:
softmax_mask = torch.nn.functional.softmax(lm_logits[0], dim=1)
lm_label = torch.argmax(softmax_mask, dim=1)

In [34]:
lm_label.shape
lm_label

tensor([ 1105,  2628,  1114,   190,  9324,  1616, 14441,  8974,  1105,   172,
         3161,  1197, 15342,  1548,  1104,  1103, 11911,   119,  1607,  1104,
         1675,  6946,  1103,  5351,  1110,   170,  2581,  1150,  1125,   170,
         1607,  1104,  1884,  4934,  4182,   119])

In [35]:
' '.join(tokenizer.convert_ids_to_tokens(lm_label))

'and associated with u ##rina ##ry tract infection and c ##ir ##r ##hos ##is of the liver . history of present illness the patient is a male who had a history of co ##lon cancer .'

In [36]:
torch.argmax(torch.nn.functional.softmax(nsp_logits, dim=1))

tensor(1)

In [98]:
import torch
sentence1 = 'associated diagnosis urinary tract infection and cirrhosis of the liver'
sentence2 = 'history of present illness the patient is an male who had a history of colon cancer.'
encoding = tokenizer(sentence1, sentence2, return_tensors='pt')
outputs = nsp_model(**encoding, labels=torch.LongTensor([1]))
logits = outputs.logits

In [99]:
outputs

NextSentencePredictorOutput(loss=tensor(0.0792, grad_fn=<NllLossBackward0>), logits=tensor([[-0.9300,  1.5663]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [100]:
logits[0,0] < logits[0,1]

tensor(True)

In [104]:
torch.argmax(torch.nn.functional.softmax(outputs.logits[0], dim=0))

tensor(1)

In [25]:
import tensorflow as tf
filename = './clinicalBERT-master/lm_pretraining/out/n2c2_train.tfrecord'
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

ModuleNotFoundError: No module named 'tensorflow'

In [48]:
tokenizer.convert_ids_to_tokens([101, 178, 102, 103, 102])
tokenizer.convert_ids_to_tokens([1117])

['his']

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from datasets import ClassLabel
import torch
from torch.utils.data import DataLoader
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, TrainingArguments, DataCollatorForLanguageModeling, Trainer

datasets = ['This is my first sentence.', 'This is the second sentence.', 'What about the third?'] * 100

In [ ]:
# datasets

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", output_attentions=True)

In [ ]:
inputs = tokenizer(datasets)

In [ ]:
tokenizer.decode(inputs['input_ids'][0])

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)
train_dl = DataLoader(inputs, batch_size=3, collate_fn=data_collator)

In [ ]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

In [ ]:
next(iter(train_dl))

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
lm_dt

In [ ]:
lm_dt

In [ ]:
show_random_elements(datasets['train'])

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

block_size = tokenizer.model_max_length
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


model_checkpoint = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
# tokenizer.convert_ids_to_tokens(tokenized_datasets['train']['input_ids'][3])

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4)

In [ ]:
tokenizer.decode(lm_datasets['train'][10]['input_ids'])

In [ ]:
datasets['train'][10]['text']

In [ ]:
tokenizer.decode(lm_datasets['train'][10]['labels'])

In [ ]:
lm_datasets.keys()

In [ ]:
lm_dt = DatasetDict({'train': lm_datasets['train'][:2], 'validation': lm_datasets['validation'][:2], 'test': lm_datasets['test'][:2]})

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
model

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dt["train"],
    eval_dataset=lm_dt["validation"],
    data_collator=data_collator)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, AutoTokenizer, AutoModel, AutoModelForMaskedLM, BertForMaskedLM, BertTokenizer, BertModel, DataCollatorForLanguageModeling

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForMaskedLM.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", output_attentions=True)

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMaskedLM.from_pretrained("bert-base-uncased")

In [ ]:
raw_datasets = ['This is my first sentence.', 'This is my second sentence.', 'This is my third sentence.']

In [ ]:
out = tokenizer(raw_datasets, return_tensors='pt', padding=True)

In [ ]:
out

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)
train_dl = DataLoader(out, batch_size=3, collate_fn=data_collator)

In [ ]:
for batch in train_dl:
    break
print({k: v for k, v in batch.items()})

In [ ]:
batch

In [ ]:
tokenizer.convert_ids_to_tokens(out["input_ids"][1])

In [ ]:
res = model(**train_dl)

In [ ]:
model.train()

In [ ]:
out.output

In [ ]:
tokenizer.convert_ids_to_tokens(out["input_ids"][0])

In [ ]:
res[0]

In [ ]:
tokenizer.convert_ids_to_tokens(torch.argmax(res[0][0], dim=-1))

In [ ]:
tokenizer.vocab_size

# Redundancy Investigation

In [ ]:
import pickle as pkl
import pandas as pd
import csv
import numpy as np
import re
import spacy
import nltk
from IPython.display import display, Markdown
from collections import namedtuple
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from spacy import Language
nlp = spacy.load('en_core_sci_md', disable=['ner'])

In [ ]:
prova = """Admission Date:  [**2174-4-18**]              Discharge Date:   [**2174-5-17**]

Date of Birth:  [**2135-11-15**]             Sex:   F

Service: MEDICINE

Allergies:
Prochlorperazine / Heparin Agents

Attending:[**First Name3 (LF) 3918**]
Chief Complaint:
Abdominal Pain"""

for t in re.finditer('\[\*\*.+?\*\*\]', re.sub('  +', ' ', prova.replace('\n', ' '))):
    print(t)

In [ ]:
re.sub('  +', ' ', prova.replace('\n', ' '))

In [ ]:
note = """
470971328 | AECH | 09071283 | | 6159055 | 5/26/2006 12:00:00 AM | PNUEMONIA | Signed | DIS | Admission Date: 4/22/2006 Report Status: Signed

Discharge Date: 7/27/2006
ATTENDING: CARINE , WALTER MD
SERVICE:
Medicine Service.
ADMISSION INFORMATION AND CHIEF COMPLAINT:
Hypoxemic respiratory failure.
HISTORY OF PRESENT ILLNESS:
The patient is a 57-year-old woman with a past medical history of
OSA , asthma , CAD status post CABG. On 8/19/06 , she underwent a
right total knee replacement at Dola Elan Hospital .  On
8/9/06 , she was discharged to rehabilitation. There , she
experienced fever , cough and dyspnea. She was started on
vancomycin , ceftazidime , and Flagyl for presumed pneumonia. In
the L ED , the patient was afebrile with a temperature of 97.6 ,
pulse of 88 , blood pressure 117/70 , oxygen saturation 97% on 6
liters nasal cannula. Her exam was notable for crackles in the
left base and 1+ lower extremity edema.
ADMISSION LABS:
Notable for white blood cell count of 20 , hematocrit 3of 5 ,
platelets of 442 , 000 , creatinine of 0.6 , and INR of 1.2. Her
admission EKG revealed sinus tachycardia of 119 beats per minute ,
normal axis , QRS 104 milliseconds , QTC 461 milliseconds , no
evidence of atrial enlargement or ventricular hypertrophy , poor
R-wave progression , 2 mm ST depressions and T-wave inversions in
leads 1 , aVL , V5 , V6 , 1 mm J-point elevation in V3 ( prior EKG
showed T-wave inversions in 1 , and aVL with no ST depressions ).
Her admission chest x-ray revealed bilateral diffuse patchy
opacities.
The patient was presumed to have pneumonia versus CHF. She was
treated with vancomycin , cefotaxime , levofloxacin , and
azithromycin , and was admitted to the Medicine Service for
further evaluation and management.
PAST MEDICAL HISTORY:
1. Left carotid artery stenosis status post CEA.
2. Right carotid artery stenosis , status post angioplasty.
3. OSA.
4. Asthma.
5. CAD status post three-vessel CABG in 2004 and subsequent PCI
to the ramus in 2005.
6. 70-80% RCA stenosis not bypassed during CABG.
7. Hypertension.
8. CHF , ejection fraction 45-50%.
9. AS status post aortic valve replacement.
10. Pericarditis removal.
11. Diabetes.
12. Peripheral vascular disease.
MEDICATIONS AT REHAB:
1. Vancomycin 1 gram IV q. 12h. , ( first dose 27 of March ).
2. Ceftazidime 1 g IV q. 8h. , ( first dose 7/17/06 )
3. Flagyl 500 mg IV q. 8h. , ( first dose 7/17/06 .
4. Advair 100/50 inhaled b.i.d.
5. Aspirin 325 mg p.o. daily.
6. Lipitor 80 mg p.o. at bedtime.
7. Zetia 10 mg p.o. daily.
8. Lopressor 75 mg p.o. q. 6h.
9. Lasix 1 tablet p.o. daily.
10. Colace 100 mg p.o. b.i.d.
11. Multivitamin 1 tab p.o. daily.
12. CaCO3 500 mg p.o. daily.
13. Cholecalciferol 400 units p.o. daily.
14. Ferrous sulfate 300 mg p.o. t.i.d.
15. Folic acid 1 mg p.o. daily.
16. Avapro 225 mg p.o. daily.
17. Lantus 100 units subq daily.
18. Lispro sliding scale.
19. Coumadin.
20. P.r.n. oxycodone , Tylenol , Benadryl , and Metamucil.
ALLERGIES:
Lisinopril leads to cough and metformin leads to GI distress.
SOCIAL HISTORY:
The patient was formerly employed as a cashier. She has two
children. She is a former cigarette smoker. She does not use
alcohol.
FAMILY HISTORY:
The patient has a positive family history of coronary disease ,
hypertension and diabetes.
HOSPITAL COURSE BY SYSTEM/ PROBLEM:
Persistent pulmonary
1. Hypoxemic respiratory failure. On 1/7/06 , shortly after
her admission to the medical floor , the patient was noted to be
in respiratory distress with tachypnea , accessory muscle use and
oxygen saturation of 68% on 6 liters nasal cannula. She was
placed on a nonrebreather. Her oxygen saturation increased to
93%; however , she continued to be in respiratory distress with
tachypnea and accessory muscle use. She was intubated and
transferred to the Medical Intensive Care Unit for further
evaluation and management. Her respiratory failure was thought
to be secondary to pneumonia with a component of superimposed
volume overload. She was treated with a 10-day course of
vancomycin , levofloxacin and ceftazidime as well as with IV
Lasix. She underwent a code green on 7/30/06 during an ETT tube
change , wherein a patent airway was transiently loss. The
patient was slow to wean from the ventilator. Her chest imaging
revealed persistent bilateral opacifications. It was thought
that after an initial infectious insult , the patient developed
ARDS. On 9/14/06 , the patient underwent bronchoscopy and BAL
revealing MRSA and HSV. The patient was treated with a 10-day
course of acyclovir for presumed HSV tracheobronchitis. Given
her inability to be weaned from the vent , the patient underwent a
tracheostomy on 4/17/06 .  Post-tracheostomy , the patient
alternated between pressure support ventilation with low driving
pressure and PEEP with a trach collar..
Infectious disease:
1. Fevers. From 1/7/06 to 7/28/06 , the patient was treated
with vancomycin , levofloxacin and ceftazidime for hospital
acquired pneumonia. After discontinuation of her antibiotics ,
the patient continued to spike fevers and evidence of
leukocytosis. In verification into source of her fevers included
serial blood cultures , urine cultures and C-dif. Positive data
included: 7/28/06 , urine culture with yeast , 7/28/06 , blood
culture with coag-negative staph , 9/14/06 BAL washings with HSV ,
9/14/06 , blood culture with coag-negative staph , 7/11/06 , blood
culture with coag-negative staph , 2/29/06 , urine culture with
yeast , 2/29/06 BAL washings with MRSA , 10/28/06 urine with yeast
urine with yeast , 10/28/06 , 6/23/06 , 4/17/06 , and 10/11/06
sputum with MRSA. 7/28/06 , chest CT with bilateral
opacification in the lung parenchyma. 2/29/06 facial CT with
left sphenoid maxillary thickening. 2/29/06 chest CT with
bilateral opacification in the lung parenchyma. Of note , a
2/29/06 abdominal CT showed no evidence of abdominal infection ,
2/29/06 TTE showed no obvious vegetations , and 10/11/06 tap of
the right knee grew no organisms. In light of the data above ,
the patient&apos;s indwelling catheters were changed. She underwent
treatment with linezolid x7 days for MRSA line infection. She
also underwent treatment with acyclovir x10 days for HSV
tracheobronchitis. The aforementioned antibiosis was mostly
prophylactic. It was thought that the patient&apos;s intermittent
fevers were not infectious , but rather reflected a drug allergy ,
most likely to vancomycin. This hypothesis was supported by a
robust eosinophilia coinciding with vancomycin administration.
Shortly after vancomycin discontinuation , the patient&apos;s fevers
resolved. The patient was afebrile for greater than 48 hours off
all antibiotics prior to transfer to rehabilitation.
Cardiovascular:
1. Volume status. The patient&apos;s admission weight was 106.2 kg.
It is unclear what her dry weight was. Given that pulmonary
edema was thought to be contributing to the patient&apos;s slow
ventilator wean , she was diuresed with a combination of Lasix and
Diuril followed by combinations of torsemide and Diuril. Her
discharge weight was 100.7 kilograms. Her diuretic regimen on
discharge with torsemide 100 mg IV t.i.d. and Diuril 500 mg IV
t.i.d. The patient&apos;s diuretic regimen will need be adjusted as
her intake is adjusted. Her creatinine will need to be monitored
very closely. Her weight will need to be checked daily.
2. Pump: The patient underwent echocardiogram on 11/19/06 ,
7/17/06 , and 2/29/06 .  On the whole , these studies revealed an
ejection fraction of 45-50% , concentric LVH , global hypokinesis
with regional variation , mild left atrial enlargement , mild
tricuspid regurgitation , a question of mild atrial stenosis , and
pulmonary artery pressures in the 40s. For her heart failure ,
the patient was treated with Lopressor and diuretics as above.
She was not started on an ACE inhibitor given her allergy
( cough ). She was started on low-dose ARB.
3. Ischemia: The patient has a history of coronary artery
disease status post three-vessel CABG and subsequent
single-vessel PPI. She has an RCA stenosis , 70-80% that has not
intervened upon. On admission , in the setting of respiratory
distress , the patient was in sinus tachycardia with rate related
to lateral ST depressions. Her cardiac biomarkers were positive
consistent with NSTEMI. It was thought that the patient
experienced demand ischemia rather than an acute plaque rupture.
On 10/8/06 , her troponin peaked at 8.53 , her CK at 275 , and her
MB of 16.3. The patient was treated with aspirin , Lopressor , and
Zocor. She was not started on an ACE inhibitor as detailed
above. She will likely warrant Cardiology followup with possible
RCA revascularization.
Neuro:
1. Sedation: While intubated , the patient was treated with IV
Versed and fentanyl titrated to light sedation. After her
tracheostomy , the patient&apos;s Versed and fentanyl drips were
discontinued. She was treated with Seroquel at bedtime to
preclude nighttime agitation. Her QTC should be monitored while
on Seroquel.
GI:
1. FEN: The patient initially received tube feeds via feeding
tube. She underwent a PEG placement on 4/17/06 .  She continued
on tube feeds. She also passed speech and Swallow and was thus
started on p.o. feeds with aspiration precautions. She also
received supplemental multivitamins , calcium carbonate , and
cholecalciferol. The patient will require speech and swallow
evaluation at rehabilitation. Now that she is awake , she may be
able to tolerate oral feeds with aspiration precautions.
2. Bowel regimen. The patient was treated with Colace , senna ,
and Dulcolax.
Heme:
1. Anemia: The patient has known iron deficiency anemia. She
was continued on iron and folate. She may benefit from an
outpatient colonoscopy if she has not had one recently.
2. Bleeding from tracheostomy site: On 4/24/06 , the patient
was noted to have bleeding from her tracheostomy site. She had
no hematocrit drop. She remained hemodynamically stable. She
underwent a bronchoscopy , which showed no active bleeding. Her
mild bleeding was thought to relate to suction trauma. Her
prophylactic heparin was held x1 day. It was thought that the
patient should be discharged on prophylactic heparin because her
DVT risk is so high. Should the patient have intense bleeding
from the tracheostomy site , a hematocrit drop or hemodynamic
changes. Her heparin subcu should be discontinued , her
hematocrit should be monitored closely , and she should be
transfused as needed. She should also at that point probably
undergo reevaluation by Pulmonary or Thoracics.
Endocrine:
1. Diabetes: The patient was treated with Lantus plus regular
insulin q. 6h. plus sliding scale insulin while she was on tube
feeds. Her insulin was changed on the night prior to discharge.
She was started on Lantus 100 subq b.i.d. She got her first dose
of 100 units subq on the evening prior to discharge , her morning
sugars were in the mid 100s. Her blood sugars should be followed
closely on the first one or two days at rehab. The blood sugars
should be monitored every 2-3 hours and her insulin should be
adjusted accordingly. Her insulin dose should be adjusted if her
tube feeds are cycled rather than given continuously or if she is
NPO.
Ortho:
1. Total knee replacement: The patient is status post right
total knee replacement on 8/19/06 .  Her right knee has a
well-healed incision and is not erythematous or tender. The
patient was initially on low-dose anticoagulation with Coumadin.
Her Coumadin was discontinued given her acute illness and her
need for procedures. She was treated with heparin subq
prophylactic doses 5000 units t.i.d. as described above. She
will need rehabilitation for her knee.
2. Prophylaxis: The patient was treated with subcutaneous
heparin and Nexium.
3. Access: The patient has peripheral IVs.
DISCHARGE STATUS:
On the day of discharge 3/8/06 , the patient was afebrile.
Heart rate was in the 80s , blood pressure was in the
100s-130s/60s-80s. Discharge weight was 100.7 kilograms. The
patient was drowsy , but arouseable. She was breathing on a trach
collar. She had decreased breath sounds at the bases. She had
an S1 and S2 with a 2/6 systolic murmur at the lower sternal
border. Her abdomen was soft and nontender with positive bowel
sounds. Her trach and PEG site were intact without any
surrounding erythema. Her extremities were warm without edema.
She was on trach collar with 50% FiO2 and oxygen saturation of
97%. Her discharge labs included a white blood cell count of 21 ,
stable , hematocrit of 28 , and platelets 436 , 000. Her chem-7
included a sodium of 131 , potassium 3 , chloride 87 , CO2 32 , BUN
67 , creatinine 0.8 , and glucose 154. Her LFTs were normal. Her
INR was 1.1. Her most recent microdata showed sputum with few
polys and a few gram-positive cocci in clusters. Her chest x-ray
showed a trach in place and bilateral hazy infiltrate consistent
with resolving ARDS.
An addendum will be given with discharge medications and doses.
At rehab , the patient&apos;s weight should be monitored daily. Her
blood sugars initially be monitored every 2-3 hours. After that ,
her blood sugar monitoring can be spaced out to t.i.d. or
whenever deemed appropriate by the physician at the
rehabilitation facility. Her hematocrit should be checked
particularly if she has any bleeding from her tracheostomy site;
otherwise , the hematocrit can be checked every day. The white
blood cell count should be monitored if the patient has any
fever. The creatinine should be monitored daily in light of the
patient&apos;s changing diuretic regimen. The patient in&apos;s and out&apos;s
should be monitored closely.
CONTACTS AT THE HOSPITAL:
Liri Hospital Of regarding the patient&apos;s inpatient course or
Dr. Julio Golding . Contacts regarding follow up issues , the
patient&apos;s primary care physician , Dr. Adrian Mckinley Rickenbaker at Mawahutche Rehabilitation Hospital Of 080-524-3286. The patient&apos;s healthcare proxy is her
sister , Kyle Lusk who is a registered nurse , her telephone
number 507-122-9398 at home and 581-981-9887 cell phone.
eScription document: 9-3627813 EMSSten Tel
CC: Geraldo Lanny Bogda MD
Los Di Wi
CC: Raleigh Semmens MD
Dictated By: DELIBERTIS , BRADLY
Attending: BEAGLEY , ORLANDO
Dictation ID 4434301
D: 3/8/06
T: 3/8/06
"""

In [ ]:
note = """
Admission Date:  [**2174-4-18**]              Discharge Date:   [**2174-5-17**]

Date of Birth:  [**2135-11-15**]             Sex:   F

Service: MEDICINE

Allergies:
Prochlorperazine / Heparin Agents

Attending:[**First Name3 (LF) 3918**]
Chief Complaint:
Abdominal Pain

Major Surgical or Invasive Procedure:
Upper GI series with small bowel follow through
Right heart catheterization
IR guided paracentesis


History of Present Illness:
38 yo F w/ h/o ALL in remission s/p cord transplant in [**1-13**],
anthracycline-induced cardiomyopathy (EF 15-20% [**1-14**]) and
recurrent nausea and vomiting who presents with abdominal pain,
N/V x1 week

Of note, the pt was admitted here from [**Date range (1) **] with nausea and
vomitting of unclear etiology. When discharged, she was
tolerating good PO and had planned f/u with neuro for ?
abdominal migraine and GI for possible other contributing
factors including food sensitivities and gastroparesis.

In the ED, VS: 98.8 94 138/100 16 100% and [**10-15**] pain. CT A/P
showed a small umbilical hernia; interval increase in size and
mild fat stranding and interval increase in ascites compared to
recent prior imaging.  WBC 12.4 with no left shift, bili 2.1 up
from 1.1, Cr 2.7 up from 2.3.  Surgery was consulted give CT
finding and did not feel there was an indication for surgery.
She received iv zofran and morphine 4mg iv and 1L IVF.

On arrival to the floor, patient reports [**11-14**] total body pain
and nausea.  She has had ice chips today but threw them up in
the ED.

Review of Systems:
(+) Per HPI
(-) Review of Systems: Denies fevers, chest pain, SOB, diarrhea,
constipation, dysuria, HA, change in vision or dizziness.


Past Medical History:
ONCOLOGIC HISTORY:
ALL:
- initially presented in [**2172-8-5**] right chest and right upper
extremity pain and paresthesias and visual blurriness. WBC
149,000; received leukapheresis, started on hydroxyurea. Dx'ed
with precursor B-cell ALL.
- underwent phase I induction with daunorubicin, vincristine,
dexamethasone, L-asparaginase, MTX; phase II with
cyclophosphamide, cytarabine, mercaptopurine, MTX
- Bone Marrow Aspirate/Biopsy on [**2172-10-26**] showed no morphologic

evidence of residual leukemia
- underwent allo double cord blood SCT [**2173-1-11**], course
complicated by neutropenic fever and acute skin GVHD

OTHER MEDICAL HISTORY:
- Embolic stroke in [**3-/2174**] on coumadin
- Cardiomyopathy due to early anthracycline-related
cardiotoxicity [**10/2172**]
- Chronic kidney disease stage III/IV, baseline creatinine
~2.0-2.2
- Asthma
- HTN
- Cervical Intraepithelial neoplasia
- C-section in [**2165**]


Social History:
Smoke: never
EtOH: Occasional in past, none currently
Drugs: Never
Lives/works: Single, has two children (ages 7 and 18). Lives in
[**Location 686**]. Was previously employed at [**Company 59330**], hasn't been
working since being diagnosed with ALL in [**2172-8-5**].


Family History:
Mother with gastric cancer, passed at the age of 40
Father with HTN.


Physical Exam:
VS: 98 145/76 87 15 100% RA
GEN: well appearing F in NAD
HEENT: slight dry MM, sclera anicteric, PERRL
Cards: RR S1/S2 normal. prominent S3
Pulm: CTAB
Abd: Hyperactive BS.  Initially soft when palpating with
stethoscope over all 4 quadrants then suddenly exquisitely
tender on right.  No guarding initially.  Unable to assess for
HSM.
Extremities: wwp, no edema. PTs 2+.
Neuro: CNs II-XII grossly intact. normal gait
Psych:  overly dramatic affect


Pertinent Results:
On admission:
[**2174-4-18**] 02:00PM BLOOD WBC-12.4* RBC-3.78* Hgb-11.4* Hct-36.3
MCV-96 MCH-30.2 MCHC-31.4 RDW-16.5* Plt Ct-212
[**2174-4-18**] 02:00PM BLOOD Neuts-67.3 Lymphs-23.8 Monos-7.7 Eos-0.5
Baso-0.7
[**2174-4-18**] 04:30PM BLOOD PT-30.1* PTT-29.4 INR(PT)-3.0*
[**2174-4-18**] 02:00PM BLOOD UreaN-30* Creat-2.7* Na-142 K-4.8 Cl-99
HCO3-31 AnGap-17
[**2174-4-18**] 02:00PM BLOOD ALT-15 AST-18 AlkPhos-127* TotBili-2.1*
[**2174-4-18**] 02:00PM BLOOD Lipase-63*
[**2174-4-18**] 02:00PM BLOOD cTropnT-<0.01
[**2174-4-18**] 02:00PM BLOOD Albumin-3.8 Calcium-9.3 Phos-4.8* Mg-2.0

On discharge:
[**2174-5-17**] 12:00AM BLOOD WBC-19.1* RBC-3.86* Hgb-11.3* Hct-37.7
MCV-98 MCH-29.3 MCHC-30.0* RDW-17.8* Plt Ct-419
[**2174-5-17**] 12:00AM BLOOD Neuts-81.3* Lymphs-11.4* Monos-6.9
Eos-0.1 Baso-0.3
[**2174-5-17**] 12:00AM BLOOD PT-31.2* PTT-28.6 INR(PT)-3.1*
[**2174-5-17**] 12:00AM BLOOD Fibrino-162
[**2174-5-17**] 12:00AM BLOOD Glucose-152* UreaN-78* Creat-2.9* Na-137
K-4.7 Cl-95* HCO3-31 AnGap-16
[**2174-5-17**] 12:00AM BLOOD ALT-51* AST-41* LD(LDH)-327* AlkPhos-107*
TotBili-0.7
[**2174-5-13**] 12:11PM BLOOD cTropnT-<0.01
[**2174-5-17**] 12:00AM BLOOD Albumin-3.8 Calcium-8.7 Phos-2.1* Mg-2.7*
UricAcd-8.7*
[**2174-4-27**] 02:51AM BLOOD calTIBC-246* Ferritn-107 TRF-189*
[**2174-5-2**] 05:55AM BLOOD [**Doctor First Name **]-NEGATIVE dsDNA-NEGATIVE
[**2174-4-28**]  HHV-8 DNA, QL PCR       Not Detected
[**2174-4-27**] QUANTIFERON(R)-TB GOLD        NEGATIVE
NEGATIVE
[**2174-4-29**] ACE, SERUM                    30                  [**10/2130**]
U/L

Micro:
[**2174-4-25**] 1:07 pm PERITONEAL FLUID
   GRAM STAIN (Final [**2174-4-25**]):
      NO POLYMORPHONUCLEAR LEUKOCYTES SEEN.
      NO MICROORGANISMS SEEN.
   FLUID CULTURE (Final [**2174-4-28**]):    NO GROWTH.
   ANAEROBIC CULTURE (Final [**2174-5-1**]):    NO GROWTH.
   ACID FAST CULTURE (Preliminary):    NO MYCOBACTERIA ISOLATED.

   ACID FAST SMEAR (Final [**2174-4-30**]):
      NO ACID FAST BACILLI SEEN ON DIRECT SMEAR.
   FUNGAL CULTURE (Final [**2174-5-13**]):    NO FUNGUS ISOLATED.

[**2174-4-29**] 10:15 pm BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)
   BLOOD/FUNGAL CULTURE (Preliminary):    NO FUNGUS ISOLATED.
   BLOOD/AFB CULTURE (Preliminary):    NO MYCOBACTERIA ISOLATED.


CMV Viral Load (Final [**2174-5-6**]): CMV DNA not detected.

ECG [**2174-4-18**]:
Sinus rhythm. Possible left atrial abnormality. Lateral ST-T
wave
abnormality. Cannot rule out myocardial ischemia. Poor R wave
progression.  Cannot rule out anterior wall myocardial
infarction of indeterminate age. Compared to the previous
tracing of [**2174-4-2**] multiple described abnormalities persist.

CT abdomen/pelvis without contrast [**2174-4-18**]:
FINDINGS: There is a small-to-moderate right pleural effusion,
smaller in
size compared to last CT torso. There is a small pericardial
effusion. Study is suboptimal for evaluation of solid organs due
to lack of IV contrast. With this limitation in mind, there is
no extra- or intra-hepatic biliary duct dilatation. Previously
described presumably focal nodular hyperplasia in segment VI of
the liver is not clearly visualized on a non-contrast CT. There
is a presumably gallbladder wall edema from third spacing with
moderate amount of ascites. There is likely gallbladder sludge.
Pancreas and bilateral adrenal glands are within normal limits
considering the limitation of no contrast administration. There
is interval increase in size of a fat-containing umbilical
hernia measuring 2 cm in transverse dimension with mild fat
stranding(2:50), correlate with point tenderness/physical exam.
The appendix is not dilated (2:49), contains air and there is a
likely small appendicolith (2:53). There is no bowel
obstruction. There is no evidence of colonic wall thickening,
although evaluation is suboptimal given lack of IV or PO
contrast and adjacent ascites.. The kidneys are normal in size.
There is no evidence of hydronephrosis. Due to lack of oral
contrast, evaluation for mesenteric lymph nodes is suboptimal.
There are scattered lymph nodes in the retroperitoneum, however,
do not meet the CT criteria for pathologic enlargement.

CT PELVIS: There is free fluid in the pelvis - ascites. The
uterus and urinary bladder appear normal. The rectum and sigmoid
have scattered diverticula; however, no evidence of
diverticulitis.

OSSEOUS STRUCTURES: No suspicious lytic or sclerotic lesion.
There is soft tissue stranding suggesting anasarca.

IMPRESSION:
1. Mild-to-moderate right pleural effusion; however, interval
decrease in size compared to prior.
2. Moderate ascites with interval increase.
3. No drainable fluid collection, however, evaluation is
suboptimal due to lack of IV and oral contrast.
4. Diverticulosis.
5. Interval increase in size of a small fat-containing umbilical
hernia with mild fat stranding, correlate with point tenderness.

6. No bowel obstruction. No definite bowel wall thickening,
although the examination is suboptimal for such.
7. Pericardial effusion, similar to prior.

RUQ ultrasound [**2174-4-18**]:
FINDINGS: The liver is of normal echogenicity. Previously
described presumably focal nodular hyperplasia in segment VI of
the liver is not clearly visualized. There is no intra- or
extra-hepatic biliary duct dilatation. The common bile duct
measures 2 mm. There is ascites. There is gallbladder wall
edema/thickening presumably from third spacing; the gallbadder
is not distended. No convincing evidence of sludge on
ultrasound. The main portal vein is patent. Pancreas is
suboptimally evaluated due to overlapping bowel gas. There is a
small-to-moderate right pleural effusion as seen on recent CT.

IMPRESSION:
1. Ascites.
2. Gallbladder wall edema presumably from third spacing.
3. Small-to-moderate right pleural effusion.
4. No biliary duct dilatation.
5. Previously described presummed focal nodular hyperplasia in
segment VI of the liver is not clearly visualized.

Small bowel follow through [**2174-4-20**]:
IMPRESSION:
1. Small, anterior cervical web that does not hinder the passage
of a 13mm
barium tablet.
2. Filling defect in the mid esophagus just below the carina
appears to be either extrinsic compression versus a submucosal
lesion. In correlation with the comparison CT torso, mediastinal
lesion is less likely. Submucosal esophageal lesion remains
within the differential, and direct visualization with EGD is
recommended. Other possibility includes an aberrant vessel in
this vicinity.
3. Mobile cecum which does not appear to be obstructive in any
manner on today's examination.

Renal ultrasound [**2174-4-20**]:
FINDINGS: The right kidney measures 10.5 cm. The left kidney
measures 9.7
cm. There is no evidence of hydronephrosis, stone or mass
bilaterally. The
bladder is unremarkable. Moderate amount of ascites is
incidentally noted.

IMPRESSION: No hydronephrosis, stone or mass within the kidneys.


Peritoneal Fluid [**2174-4-25**]:
  ATYPICAL.
  Scattered atypical lymphoid cells in a background of
  reactive mesothelial cells

IR guided paracentesis [**2174-4-25**]:
IMPRESSION:
Ultrasound-guided diagnostic paracentesis, with a total of 200
mL of ascites removed.

TTE [**2174-5-2**]:
The left atrium is mildly elongated. Left ventricular wall
thicknesses and cavity size are normal. There is severe global
left ventricular hypokinesis (LVEF = 20 %). Systolic function of
apical segments is relatively preserved. No masses or thrombi
are seen in the left ventricle. Right ventricular chamber size
is mildly increased with moderate global free wall hypokinesis.
[Intrinsic right ventricular systolic function is likely more
depressed given the severity of tricuspid regurgitation.] The
aortic valve leaflets (3) appear structurally normal with good
leaflet excursion and no aortic stenosis or aortic
regurgitation. The mitral valve appears structurally normal with
trivial mitral regurgitation. There is no mitral valve prolapse.
Severe [4+] tricuspid regurgitation is seen. There is mild
pulmonary artery systolic hypertension [In the setting of at
least moderate to severe tricuspid regurgitation, the estimated
pulmonary artery systolic pressure may be underestimated due to
a very high right atrial pressure.] There is a small
circumferential pericardial effusion without echocardiographic
signs of tamponade.

IMPRESSION: Severe biventricular global hypokinesis. Severe
tricuspid regurgitation. Pulmonary artery systolic hypertension.
Small circumferential pericardial effusion without evidence of
tamponade physiology.
Compared with the prior study (images reviewed) of [**2174-4-1**],
the findings are similar.

TTE [**2174-5-10**]:
The left atrium is dilated. A left-to-right shunt across the
interatrial septum is seen at rest consistent with a stretched
patent foramen ovale (or small atrial septal defect). There is
mild symmetric left ventricular hypertrophy. The left
ventricular cavity size is normal with mildly impaired global
left ventricular systolic function. The aortic valve leaflets
(3) appear structurally normal with good leaflet excursion and
no aortic stenosis or aortic regurgitation. The mitral valve
appears structurally normal with trivial mitral regurgitation.
The tricuspid valve leaflets are mildly thickened. There is
moderate (2+) tricuspid regurgitation. There is mild pulmonary
artery systolic hypertension. There is a small pericardial
effusion. There are no echocardiographic signs of tamponade.
Echocardiographic signs of tamponade may be absent in the
presence of elevated right sided pressures.

Compared with the prior study (images reviewed) of [**2174-5-6**],
ther pericardial effusion is now smaller. Biventricular
sysotolic function appears slightly less vigorous compared to
the prior study (on a lower dose of milrinone now than during
the prior study).

Cardiac cath [**2174-5-5**]:
COMMENTS:
1. Hemodynamics measurements in this patient demonstrate low
cardiac output. Following administration of milrinone, cardiac
index increased to the low-normal range with 2.5 L/min/m2.
2. Moderate pulmonary hypertension with right atrial v-waves
consistent with severe TR noted. Pulmonary vascular resistance
is elevated at 280 dyne-cm-sec5.

FINAL DIAGNOSIS:
1. Severe systolic ventricular dysfunction.
2. Moderate diastolic ventricular dysfunction.
3. Pulmonary hypertension

LE ultrasound [**2174-5-13**]:
IMPRESSION:
1. No evidence for deep venous thrombosis in either lower
extremity.
2. 3.6 cm [**Hospital Ward Name 4675**] cyst in the right popliteal fossa as previous.
Superficial soft tissue edema in the right mid thigh, may be
related to partial rupture of [**Hospital Ward Name 4675**] cyst.

TTE [**2174-5-16**]:
The left atrium is dilated. Left ventricular wall thicknesses
and cavity size are normal. The diameters of aorta at the sinus,
ascending and arch levels are normal. The aortic valve leaflets
(3) appear structurally normal with good leaflet excursion and
no aortic stenosis or aortic regurgitation. The mitral valve
leaflets are structurally normal. There is no mitral valve
prolapse. Trivial mitral regurgitation is seen. Moderate [2+]
tricuspid regurgitation is seen. There is moderate pulmonary
artery systolic hypertension. There is a small pericardial
effusion. The effusion appears circumferential. There are no
echocardiographic signs of tamponade. Echocardiographic signs of
tamponade may be absent in the presence of elevated right sided
pressures.

Compared with the prior study (images reviewed) of [**2174-5-10**],
biventricular systolic function is slightly worse. The size of
the pericardial effusion is slightly smaller.

Brief Hospital Course:
38 yo F w/ h/o ALL in remission s/p cord transplant in [**1-13**],
anthracycline-induced cardiomyopathy (EF 15-20% [**1-14**]) and
recurrent nausea and vomiting who presents with 1 week abd pain,
acute on chronic renal failure and new hyperbilirubinemia.
Unclear unifying diagnosis.

# Acute on Chronic Abdominal Pain:  Pt noted to have significant
abdominal pain as well as increased [**Month/Year (2) 4394**] on admission.  Of
note, she had an extensive work up of her chronic abdominal pain
in the past with no clear cause.  Abdominal CT was unrevealing
for any obvious source of her pain. GI was consulted who
recommended a SBFT which did not reveal any significant
pathology.  GI recommended bentyl for antispasmodic effect.  She
was also continued on her home MS contin and IV morphine for
breakthrough.  Her pain persisted as did her [**Last Name (LF) 4394**], [**First Name3 (LF) **] the
decision was made to perform a diagnositc paracentesis under
ultrasound guidance.  200ml peritoneal fluid was removed.  This
revealed 775 WBCs, but a lymphocytic/monocytic predominance with
only 1% polys making SBP unlikely.  Fluid was sent for culture
which showed no growth and flow cytometry which showed no
evidence of ALL recurrence.  Despite lack of evidence for SBP,
she was started on zosyn empirically which was stopped on [**5-2**].
She continued to have mild-moderate abdominal pain but was able
to eat full meals and had BMs. She was continued on her home
mscontin and morphine IR.
.
# Anthracycline-induced/ GVHD cardiomyopathy:  EF <20% on echo
from 2/[**2174**].  Pt was maintained on diuresis as above, which was
subsequently held in the setting of rising creatinine with
improvement in creatinine. Torsemide was slowly reintroduced and
uptitrated to 40mg [**Hospital1 **] which caused another bump in creatinine
to 3.0, so renal and cardiology were consulted. Renal ultrasound
was unrevealing.  She was then taken to the Cath lab and placed
on a milrinone/lasix gtt and transfered to the CCU. Her volume
overload slowly improved and her peripheral edema/ascites slowly
improved as well. A repeat echo showed improved EF to 40-45% on
the milrinone gtt. She was then started on solumedrol 30mg IV
due to a concern for GVHD directed towards myocardium.  After
further discussion between cardiology and her oncology team she
was also started on cellcept for further management of her GVHD.
 She did well on milrinone and lasix drip, but the drip was
stopped when her creatinine bumped to 3.0 and it was felt her
volume status was near maximization.  Her milrinone was then
discontiued and she was then transferred back to [**Hospital1 3242**] for further
management of her abdominal pain and GVHD.  She was continued on
torsemide for diuresis with close follow-up with her outpatient
cardiologist.  Of note, she had frequent alarms on telemetry for
tachycardia that cardiologist felt was mostly due to artifact;
her beta blocker was uptitrated.  Repeat TTE prior to discharge
showed an EF of 35-40%.  She was discharged home on cellcept and
prednisone for possible GVHD.

# Acute Renal Failure:  On admission Cr was 2.7 (recent baseline
was 2), but at last discharge Cr was 2.3.  Renal saw the patient
who thought this was likely from overdiuresis (home torsemide
regimen of 20mg [**Hospital1 **]) in conjunction with her [**Last Name (LF) **], [**First Name3 (LF) **] recommended
holding diuresis. Her Cr subsequently improved, but in the
setting of her worsening [**First Name3 (LF) 4394**] and her cardiomyopathy,
decision was made to slowly add back diuresis, and eventually
she was up titrated to toresemide 40mg [**Hospital1 **] and her [**Last Name (un) **] was
restarted.  With this, however, her Cr began to climb again to
3.0.  Given the delicate balance between her renal failure
cardiomyopathy, cardiology/renal were consulted. Given her
depressed EF, her rising Cr was thought to be [**3-9**] volume
overload. She was sent to the cath lab and started on a
milrinone/lasix gtt and transfered to the CCU with a goal
diuresis of 1L per day. She was actively diuresed on her
milrinone and lasix drip with a total net negative of close to
9L.  Her Cr then returned to baseline by time of discharge and
she was discharged home on torsemide.

# Hyperbilirubinemia:  Unclear cause, could have been related to
a viral infection but no transaminitis to support this.  RUQ u/s
without cause for pain. This trended down to normal values and
remained stable by time of discharge

# Leukocytosis: patient had uptrending WBC in setting of
starting solumedrol, clutures were sent which revealed no
growth.
.
# H/O Embolic Stroke:  Has new opening of PFO based on most
recent echo which likely contributed to her recent stroke.  She
was maintained on coumadin 4mg daily, but anticoagulation was
held on day of paracentesis and remained subtherapeutic for
several days, so she was maintained on a heparin drip to bridge
her to a therapeutic INR [**3-10**]. She was maintained on a decreased
dose of coumadin throughout hospital admission with INR within
goal between 2 and 3. She was arranged with follow-up at
outpatient [**Hospital3 **].

Medications on Admission:
Carvedilol 25 mg [**Hospital1 **]
Fluticasone-salmeterol [**Hospital1 **]
Morphine 15 mg q6h prn pain
Valsartan 40 mg qd
Torsemide 20 mg [**Hospital1 **]
Multivitamin qd
Albuterol prn
Lorazepam 0.5 mg q6h prn nausea
Warfarin 4 mg qd
Ondansetron 8 mg tid prn
Pentamidine 300 mg inhalation qmonth
Colace 100 mg qd prn


Discharge Medications:
1. fluticasone-salmeterol 250-50 mcg/dose Disk with Device Sig:
One (1) Disk with Device Inhalation [**Hospital1 **] (2 times a day).
2. docusate sodium 100 mg Capsule Sig: One (1) Capsule PO BID (2
times a day).
3. lorazepam 0.5 mg Tablet Sig: One (1) Tablet PO every six (6)
hours as needed for nausea.
Disp:*60 Tablet(s)* Refills:*0*
4. albuterol sulfate 90 mcg/Actuation HFA Aerosol Inhaler Sig:
Two (2) Puff Inhalation Q4H (every 4 hours) as needed for sob or
wheeze.
5. Zofran 8 mg Tablet Sig: One (1) Tablet PO every eight (8)
hours as needed for nausea.
6. multivitamin     Tablet Sig: One (1) Tablet PO once a day.
7. metoprolol succinate 100 mg Tablet Extended Release 24 hr
Sig: One (1) Tablet Extended Release 24 hr PO DAILY (Daily).
Disp:*30 Tablet Extended Release 24 hr(s)* Refills:*0*
8. morphine 15 mg Tablet Extended Release Sig: One (1) Tablet
Extended Release PO Q12H (every 12 hours).
Disp:*60 Tablet Extended Release(s)* Refills:*0*
9. dicyclomine 20 mg Tablet Sig: One (1) Tablet PO four times a
day.
Disp:*120 Tablet(s)* Refills:*0*
10. allopurinol 100 mg Tablet Sig: One (1) Tablet PO DAILY
(Daily).
Disp:*30 Tablet(s)* Refills:*0*
11. sulfamethoxazole-trimethoprim 400-80 mg Tablet Sig: One (1)
Tablet PO DAILY (Daily).
Disp:*30 Tablet(s)* Refills:*0*
12. acyclovir 400 mg Tablet Sig: One (1) Tablet PO Q12H (every
12 hours).
Disp:*60 Tablet(s)* Refills:*0*
13. torsemide 20 mg Tablet Sig: Two (2) Tablet PO DAILY (Daily).
Disp:*60 Tablet(s)* Refills:*0*
14. simethicone 80 mg Tablet, Chewable Sig: One (1) Tablet,
Chewable PO QID (4 times a day) as needed for abdominal pain or
gas.
Disp:*120 Tablet, Chewable(s)* Refills:*0*
15. mycophenolate mofetil 500 mg Tablet Sig: Two (2) Tablet PO
BID (2 times a day).
Disp:*120 Tablet(s)* Refills:*0*
16. prednisone 20 mg Tablet Sig: Three (3) Tablet PO DAILY
(Daily).
Disp:*90 Tablet(s)* Refills:*0*
17. magnesium hydroxide 400 mg/5 mL Suspension Sig: Thirty (30)
ML PO Q6H (every 6 hours) as needed for constipation.
Disp:*500 ML(s)* Refills:*0*
18. morphine 15 mg Tablet Sig: One (1) Tablet PO every twelve
(12) hours as needed for pain.
19. warfarin 2 mg Tablet Sig: One (1) Tablet PO once a day.
Disp:*30 Tablet(s)* Refills:*0*


Discharge Disposition:
Home

Discharge Diagnosis:
Primary:
-Abdominal Pain
-Acute on chronic renal failure
-Systolic Heart failure

Secondary:
-ALL
-History of embolic stroke


Discharge Condition:
Mental Status: Clear and coherent.
Level of Consciousness: Alert and interactive.
Activity Status: Ambulatory - Independent.


Discharge Instructions:
Ms. [**Known lastname **],

You were admitted to the hospital for abdominal pain.  Your pain
was treated with pain medications, and a new medication called
Bentyl.  You were also switched to a longer acting form of your
morphine.  We did a test to look at your small bowel which was
negative.  At this point we are not sure what is causing your
pain, but you had increased swelling of your abdomen which
likely contributed to your pain.

You underwent a right heart catheterization and [**Known lastname 461**]
to assess your heart function because worsening heart failure
can cause fluid in your belly and worsening kidney disease.  You
were at the cardiac intensive care unit and placed on a
medication that improved your heart function.  A repeat
[**Known lastname 461**] prior to your discharge showed that your heart
function has improved somewhat and is stable.  You will follow
up closely with your cardiologist as several of your heart
medications have changed.  You were started on steroids and
mycophenolate mofetil because it was felt that you heart
problems may be due to your leukemia.

You also had some worsening of your renal failure.  You were
followed by our kidney consult team while you were in the
hospital.  Your kidney function was stable prior to discharge.

We made the following changes to your medications:
-Mycophenolate Mofetil 1000mg twice a day was started
-Prednisone 60mg daily was started
-Coumadin was decreased to 2mg daily
-Torsemide was increased to 40mg daily
-Please hold your valsartan until you see your cardiologist
-Metoprolol succinate 100mg daily was started; please stop
carvedilol
-Bentyl (dicyclomine) was started for your abdominal pain
-Simethicone was started for abdominal discomfort/gas
-Your morphine was switched to long-acting Morphine 15mg twice a
day
-Bactrim single strength, 1 tablet daily, was started to help
prevent infection
-Acyclovir 400mg twice a day was started to help prevent
infection
-Allopurinol 100mg daily was started because your uric acid
levels were high

Weigh yourself every morning, [**Name8 (MD) 138**] MD if weight goes up more
than 3 lbs.

Followup Instructions:
You have the following appointments [**Name8 (MD) 1988**] for you.  You will
need to follow up at [**Hospital3 **] on Thursday,
[**2174-5-19**], for an INR (coumadin level) check.  Please come to
the [**Hospital Ward Name 23**] Center [**Location (un) 895**] for this lab test between 9am and
5pm.

Department: HEMATOLOGY/ONCOLOGY
When: FRIDAY [**2174-5-20**] at 3:30 PM
With: [**Name6 (MD) **] [**Name8 (MD) **], MD [**Telephone/Fax (1) 22**]
Building: [**Hospital6 29**] [**Location (un) **]
Campus: EAST     Best Parking: [**Hospital Ward Name 23**] Garage

Department: HEMATOLOGY/ONCOLOGY
When: FRIDAY [**2174-5-20**] at 3:30 PM
With: [**First Name11 (Name Pattern1) **] [**Last Name (NamePattern1) 10565**], NP [**Telephone/Fax (1) 22**]
Building: SC [**Hospital Ward Name 23**] Clinical Ctr [**Location (un) 24**]
Campus: EAST     Best Parking: [**Hospital Ward Name 23**] Garage

[**First Name8 (NamePattern2) 449**] [**Last Name (NamePattern1) 437**] MD, Cardiology
[**Last Name (LF) 766**], [**2174-5-30**] at 11:00AM
SC [**Hospital Ward Name **] CLINICAL CTR, [**Location (un) **]

Department: WEST [**Hospital 2002**] CLINIC
When: THURSDAY [**2174-6-9**] at 10:00 AM
With: [**First Name11 (Name Pattern1) 177**] [**Last Name (NamePattern4) 720**], M.D. [**Telephone/Fax (1) 721**]
Building: De [**Hospital1 **] Building ([**Hospital Ward Name 121**] Complex) [**Location (un) **]
Campus: WEST     Best Parking: [**Street Address(1) 592**] Garage


                             [**Name6 (MD) **] [**Name8 (MD) **] MD [**MD Number(2) 3922**]

Completed by:[**2174-5-26**]
"""

In [ ]:
#setting sentence boundaries
@Language.component('sentbound')
def custom_sentencizer(doc):
    for i, token in enumerate(doc[:-1]):
#         if token.text == r' ?\.' and (doc[i+1].is_title or doc[i+1].is_upper or doc[i+1].like_num):
        if re.match(r'^ ?\.', token.text) and (doc[i+1].is_upper or doc[i+1].is_title):
            doc[i+1].is_sent_start = True
            token.is_sent_start = False
        elif re.match(r'[0-9]{1,2}\.$', token.text):
            if not doc[i-1].is_stop:
                token.is_sent_start = True
                doc[i+1].is_sent_start = False
            else:
                token.is_sent_start = False
                doc[i+1].is_sent_start = True
#             print(token.text, token.is_sent_start, doc[i+1], doc[i+1].is_sent_start)
        elif token.text == '-' and doc[i+1].text != '-':
            token.is_sent_start = True
            doc[i+1].is_sent_start = False
        else:
            doc[i+1].is_sent_start = False
#         if re.match(r'[0-9]{1,2}\.$', token.text) :
#             print(token.text, token.is_sent_start, doc[i-1], doc[i-1].is_stop)
#         print(token, token.is_sent_start)
#         if re.match("VENBONLEA HEALTH", token.text):
#             print(doc[i-1], doc[i-1].is_sent_start, token, token.is_sent_start, doc[i+1], doc[i+1].is_sent_start)
    return doc

#convert de-identification text, dates and lab results into one token
@Language.component('tkndef')
def def_tokens(doc):
    patterns = [r'\[\*\*.+?\*\*\]', ##de-identification
                r'[0-9]{1,4}[/\-][0-9]{1,2}[/\-][0-9]{1,4}', ##date
                r'[0-9]+\-?[0-9]+%?', ##lab/test result
                r'[0-9]+/[0-9]+', ##lab/test result
                r'([0-9]{1,3} ?, ?[0-9]{3})+', ##number >= 10^3
                r'[0-9]{1,2}\+', ##lab/test result
                r'[A-Za-z]{1,3}\.', ##abbrv, e.g., pt.
                r'[A-Za-z]\.([a-z]\.){1,2}', ##abbrv, e.g., p.o., b.i.d.
                r'[0-9]{1,2}h\.', ##time, e.g., 12h
                r'(\+[0-9] )?\(?[0-9]{3}\)?[\- ][0-9]{3}[\- ][0-9]{4}', ##phone number
                r'[0-9]{1,2}\.' ##Numbered lists
               ]
    for expression in patterns:
        for match in re.finditer(expression, doc.text):
            start, end = match.span()
            span = doc.char_span(start, end)
            # This is a Span object or None if match doesn't map to valid token sequence
            if span is not None:
                print("Found match:", span.text)
                with doc.retokenize() as retokenizer:
                    retokenizer.merge(span)
    return doc

nlp.add_pipe('tkndef', before='parser')
nlp.add_pipe('sentbound', before='parser')

doc = nlp(re.sub('  +', ' ', note.replace('\n', ' ')))
# new_doc = fix_deid_tokens(patterns, doc)
# expression = r"\[\*\*.{0,15}.*?\*\*\]"
# for match in re.finditer(expression, doc.text):
#     start, end = match.span()
#     span = doc.char_span(start, end)
#     # This is a Span object or None if match doesn't map to valid token sequence
#     if span is not None:
#         print("Found match:", span.text)
#     with doc.retokenize() as retokenizer:
#         retokenizer.merge(span)
        
# @Language.component('mycomp2')
# def lower_all(doc):
#     for i, tkn in enumerate(doc):
#         doc[i].text = doc[i].text.lower()
#     return doc.lower()


# nlp.add_pipe("merge_entities")

In [ ]:
for i, s in enumerate(doc.sents):
    print(s)
    for t in s:
        print(f"{t}")
    print('\n')

In [ ]:
for i, tk in enumerate(doc):
    print(tk, doc[i].is_sent_start)

In [ ]:
doc = nlp(note)
patterns = [r"\[\*\*|\*\*\]", ##de-identification
                r'[0-9]{1,4}[/\-][0-9]{1,2}[/\-][0-9]{1,4}', ##date
            r'(\+[0-9] )?\(?[0-9]{3}\)?[\- ][0-9]{3}[\- ][0-9]{4}', ##phone number
                r'[0-9]+\-?[0-9]+%?', ##lab/test result
                r'[0-9]+/[0-9]+', ##lab/test result
                r'([0-9]{1,3} ?, ?[0-9]{3})+', ##number >= 10^3
                r'[0-9]{1,2}\+', ##lab/test result
                r'[A-Za-z]{1,3}\.', ##abbrv, e.g., pt.
                r'[A-Za-z]\.([a-z]\.){1,2}', ##abbrv, e.g., p.o., b.i.d.
                r'[0-9]{1,2}h\.', ##time, e.g., 12h
               ]
for expression in patterns:
    print(expression)
    for match in re.finditer(expression, doc.text):
        print(match)
        start, end = match.span()
        span = doc.char_span(start, end)
        print(span)
        # This is a Span object or None if match doesn't map to valid token sequence
        if span is not None:
            print("Found match:", span.text)
    print('\n')

In [ ]:
doc.char_span(13872, 13884)

In [ ]:
for s in doc.sents:
    print(s)
    for t in s:
        print(t)
    print('\n')

In [ ]:
for t in doc:
    if t.like_num:
        print(t)

In [ ]:
matches

In [ ]:
doc = nlp("Hello world!")
with doc.retokenize() as retokenizer:
    retokenizer.merge(doc[0:2])

In [ ]:
for t in doc:
    print(t.text)

In [ ]:
doc = fix_deid_tokens('[**123**] blah blah blah', nlp('[**123**] blah blah blah'))

In [ ]:
for t in doc:
    print(t)

In [ ]:
notes = pkl.load(open('./data/train_n2c2_datasets_preprocessed.pkl', 'rb'))
n_sents = pkl.load(open('./data/train_n2c2_datasets_sentences_preprocessed.pkl', 'rb'))
n_words = pkl.load(open('./data/train_n2c2_datasets_tokenized_preprocessed.pkl', 'rb'))
n_words = {str(el[0]): el[1] for el in n_words}
n_sents = {el[0]: el[1] for el in n_sents}
with open('./data/train_newk_to_oldk.csv', 'r') as f:
    rd = csv.reader(f)
    next(rd)
    new_to_old = {row[0]: (row[1], row[2]) for row in rd} 

In [ ]:
count_ch = {}
for val in new_to_old.values():
    if val[1] in count_ch:
        count_ch[val[1]] += 1
    else:
        count_ch[val[1]] = 1

## Within-note redundancy

In [ ]:
wn_redundancy = namedtuple('wn_redundancy', ['note_id', 'nr_score', 'counts', 'challenge'])
wn_list = pkl.load(open('./data/train_wn_redundancy.pkl', 'rb'))

In [ ]:
wn_dic = {'note_id': [], 'nr_score': [], 
          'counts': [], 'challenge': []}
for el in wn_list:
    if el.note_id not in ["75", "653", "851447", "330501671", "103377", "197869", "102937", "328::02", "903_1", "333::02", "963", "343", "354_1", "915_1", "1080"]:
        wn_dic['note_id'].append(el.note_id)
        wn_dic['nr_score'].append(el.nr_score)
        wn_dic['counts'].append(el.counts)
        wn_dic['challenge'].append(el.challenge)
    
wn_dt = pd.DataFrame(wn_dic)

In [ ]:
wn_dt

In [ ]:
c_sen = []
c_rep = []
ch_counts = {}
ch_values = {}
for _, row in wn_dt.iterrows():
    c_sen.append(len(row.counts))
    c_rep.extend([val for val in row.counts.values()])
    if row.challenge in ch_counts:
        ch_counts[row.challenge].append(len(row.counts))
    else:
        ch_counts[row.challenge] = [len(row.counts)]
    if row.challenge in ch_values:
        ch_values[row.challenge].extend([val for val in row.counts.values()])
    else:
        ch_values[row.challenge] = list(row.counts.values())

In [ ]:
display(Markdown("<u>Number of notes with repeated sequences by challenge:"))
# Number of notes (with percentages) that include repeated sentences (>=2) by challenge group
for ch, n in zip(wn_dt.challenge.value_counts().index, wn_dt.challenge.value_counts()):
    display(Markdown(f"**{ch}**: {int(n)} ({round(int(n)/count_ch[ch]*100, 2)}%)"))
print('\n')

# Overall number of repeated sentences (median [min; max])
display(Markdown(f"Overall median ([min; max]) number of repeated sentences: "
                 f"{np.median(c_sen)} [{min(c_sen)}; {max(c_sen)}]"))
# Overall number of repetitions per sentence (median [min; max])
display(Markdown(f"Overall median ([min; max]) number of repetitions per sentence: "
                 f"{np.median(c_rep)} [{min(c_rep)}; {max(c_rep)}]"))
print('\n')
display(Markdown("<u>Median [min; max] number of sentences by challenge:"))
# By challenge (median [min; max] number of repeated sentences and their frequency)
for ch in ch_counts.keys():
    display(Markdown(f"**{ch}**: {np.median(ch_counts[ch])} [{min(ch_counts[ch])}; {max(ch_counts[ch])}]"))
display(Markdown("<u>Median [min; max] frequency by challenge:"))
for ch in ch_values.keys():
    display(Markdown(f"**{ch}**: {np.median(ch_values[ch])} [{min(ch_values[ch])}; {max(ch_values[ch])}]"))

print('\n')
display(Markdown(f"**Overall number of notes:** {wn_dt.shape[0]} ({round(wn_dt.shape[0]/len(notes)*100, 2)}%)"))

It appears that the within-note redundancy in the challenge notes is caused by:
1. C/P practice for some notes (longest);
2. Addendums (miedium length);
3. Override notes (medium length);
4. Repeated constant text (lab, headers) (shortest).

### Strategy
Drop all repeated sentences starting from the bottom of the note and the beginning of the dictionary.

## Between-note investigation

In [ ]:
bn_redundancy = namedtuple('bn_redundancy', ['sen_A', 'sen_B', 'align_score'])
bn_list = pkl.load(open('./data/train_bn_redundancy.pkl', 'rb'))

In [ ]:
bn_scores = []
bn_list_rid = {}
for k, el in bn_list.items():
    len_a = len([a for a in el.sen_A if a!='-'])
    len_b = len([b for b in el.sen_B if b!='-'])
    s_len = len_a + len_b
    bn_scores.append(el.align_score/s_len)
    if el.align_score > 0:
        bn_list_rid[k] = el
    
display(Markdown(f"Median alignment score ([min; max]): {round(np.median(bn_scores), 2)} [{round(min(bn_scores), 2)}; {round(max(bn_scores), 2)}]"))
display(Markdown(f"Median alignment score ([min; max]): {round(np.mean(bn_scores), 2)} ({round(np.std(bn_scores), 2)})"))

In [ ]:
seq_len = [len(el.sen_A) for el in bn_list.values()] + [len(el.sen_B) for el in bn_list.values()]
max_len = max(seq_len)

df_list = [[k] + [0]*max_len for k in bn_list.keys()]

In [ ]:
for n, el in enumerate(bn_list.values()):
    i = 0
    for a, b in zip(el.sen_A, el.sen_B):
        try:
            if a==b and (a!='-' and b!='-'):
                df_list[n][i+1] = 2
            elif a=='-' or b=='-':
                if df_list[n][i] == 1.5:
                    df_list[n][i+1] = 1.25
                else:
                    df_list[n][i+1] = 1.5
            else:
                df_list[n][i] == -10
            i += 1
        except IndexError:
            break

In [ ]:
df = pd.DataFrame(df_list)
df.columns = ["NOTE_ID"] + [f"W_{i}" for i in range(max_len)]

In [ ]:
df.index = df.NOTE_ID
df.drop('NOTE_ID', axis=1, inplace=True)

In [ ]:
# myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
# cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))
cmap = ['white', '#b3e6b3','#66cc66','#ffc299','#ff944d','#ff6600','#ccddff','#99bbff','#4d88ff','#0044cc','#002b80']

htmp = sns.heatmap(df, xticklabels=False, yticklabels=False, cmap=cmap)
sns.set(rc={'figure.figsize':(10,10)})
plt.ylabel('')
plt.show()

The alignment investigation suggests that no clinical note, among the longitudinal, is really copied-forward. It is possible that the amount of overlap is negligible. 

In [ ]:
plt.hist(bn_scores, bins=100)
plt.title("Standardized alignment scores")
plt.show()

In [ ]:
plt.title("Raw alignment scores")
plt.hist([el.align_score for el in bn_list.values()], bins=100)
plt.show()

For now let's stop here the investigation and implement the transformer. I'll get back to this when I figure out what I need to fine-tune the model (also in terms of challenge tasks).

In [ ]:
# Check the notes that are highly aligned (score > 0.7)
for k, el in bn_list.items():
    if el.align_score/len(el.sen_A) >= 0.7:
        print(k)
        

In [ ]:
chk_note = bn_list['292::1::3']
for a, b in zip(chk_note.sen_A, chk_note.sen_B):
    print(a, b)

In [ ]:
notes_dict['292::01']

In [ ]:
notes_dict['292::03']

In [ ]:
import nltk
import spacy
# nlp = spacy.load('en_core_sci_md', disable=['ner'])
doc = nlp('2094-03-14 ciao ciao. ciao ciao')

print(nltk.tokenize.word_tokenize('2094-03-14'))
print([t for t in s for s in doc.sents])

In [ ]:
for s in doc.sents:
    print(nltk.tokenize.word_tokenize(str(s)))

In [ ]:
notes_dict = {el[0]: el[1] for el in notes}

In [ ]:
notes_dict['292::01']

In [ ]:
max_val = sorted(bn_scores)[::-1][:3]
for el in bn_list:
    if el[1][-1]/len(el[1][0]) == max_val[1]:
        doc1 = el[1][1]
        doc2 = el[1][2]
        print(f"Score: {el[1][-1]/len(el[1][0])} -- Notes {el[0]} {el[1][0]}")
        for w1, w2 in zip(doc1, doc2):
            print(w1, w2)
        print('\n')

In [ ]:
max_val

In [ ]:
prova[1]